## Analyzing ADCP Data

Pulling in a multiple years of ADCP data

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

import hvplot.xarray


import cartopy.crs as crs

import pandas as pd
import geopandas as gpd
import glob
import hvplot.pandas  # noqa
from datetime import timedelta
from datetime import datetime
import scipy.io

import scipy.signal
from random import random

pull in GOES data for this full two year period via

http://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2020-09-01):1:(2022-04-15T13:00:00Z)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D

Had to split it into two downloads to finish without error:

http://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2020-09-01):1:(2021-06-15T13:00:00Z)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D

http://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2021-06-15T14:00:00Z):1:(2022-04-15T13:00:00Z)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D

this finall worked

from https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.nodc:GHRSST-ABI_G16-STAR-L3C

https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2020-09-01):1:(2021-01-01T13:00:00Z)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D

https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2021-01-01T13:00:00Z):1:(2021-04-01T13:00:00Z)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D

https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2021-04-01):1:(2021-07-15)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D

https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2021-07-15):1:(2021-10-15)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D

https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2021-10-15):1:(2022-01-01)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D

https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2022-01-15):1:(2022-03-15)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D

https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplStarG16SSTv270.nc?sea_surface_temperature%5B(2022-03-15):1:(2022-05-15)%5D%5B(37.5):1:(34.2)%5D%5B(-76.6):1:(-72.99)%5D


In [ ]:
fns = glob.glob('data/all_goes/*.nc')
goes_ds = xr.open_dataset(fns[0])

for fn in fns[1:]:
    new_ds = xr.open_dataset(fn)
    goes_ds = xr.concat([goes_ds,new_ds],dim='time')

In [ ]:
datetimeindex = goes_ds.indexes['time'].to_datetimeindex()  
goes_ds['time'] = datetimeindex
goes_ds = goes_ds.sortby('time')
goes_ds

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
import glob

all_files = glob.glob('data/all_adcp/processed/*.mat')

In [ ]:
files = []
for f in all_files:
    files.append(f.split('100s')[0])
filenames = list(np.unique(files))
filenames = [f+'100s' for f in filenames]

In [ ]:
datetimes = [datetime.strptime(f.split('_')[2][:-5], '%Y%m%dT%H%S') for f in filenames]

In [ ]:
fig,ax = plt.subplots(figsize=(12,7))
ax.scatter(datetimes, range(len(datetimes)))

In [ ]:
tentative_good_idxs = [4,5,6,7,8,9,10,11,12,13,16,17,20,21,22,23,26,27,28,30,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,57,58,59,60,61,62,63,64,65,66,67,68]

### pulling in profile code

In [ ]:
! pip install colour

In [ ]:
# yes I realize this is an ugly brute force way to do this but it works

In [ ]:
count = 0

vmp_fns = glob.glob('data/VMP/2020*.mat')
vmp_fns.sort()

names = ['t'+str(i) for i in np.arange(count,len(vmp_fns)+count)]
count += len(vmp_fns)

li = []
for i, fn in enumerate(vmp_fns):
    mat = scipy.io.loadmat(fn)
    for item in list(mat.keys()):
        if item not in ['__header__', '__version__', '__globals__']:
            df_tmp = pd.DataFrame(data=mat[item], columns=['time', 'depth (dBars)', 'temp (C)','salinity (PSU)', 'potential density (kg/m^3 -1000)', 
                                                    'chla (ppb)', 'turbidity (FTU)', 'turb kinetic energy 1 (W/kg)', 'turb kinetic energy 2 (W/kg)'])
            df_tmp['transect'] = names[i]
            df_tmp['profile_num'] = names[i]+'_'+item.split('profile')[-1]
            li.append(df_tmp)

profiles_2020 = pd.concat(li, axis=0, ignore_index=True)
profiles_2020['dt'] = pd.to_datetime('2020-1-1') + pd.to_timedelta(profiles_2020.time, unit='D') - pd.Timedelta(days=1)
profiles_2020 = profiles_2020.set_index('dt')
profiles_2020 = profiles_2020.sort_index(ascending=True)
profiles_2020['datetime'] = pd.to_datetime('2020-1-1') + pd.to_timedelta(profiles_2020.time, unit='D') - pd.Timedelta(days=1)

In [ ]:
vmp_fns = glob.glob('data/VMP/YD*.mat') + glob.glob('data/VMP/Cruise1/*.mat') + glob.glob('data/VMP/Cruise2/*.mat')
vmp_fns.sort()

names = ['t'+str(i) for i in np.arange(count,len(vmp_fns)+count)]
count += len(vmp_fns)

li = []
for i, fn in enumerate(vmp_fns):
    mat = scipy.io.loadmat(fn)
    for item in list(mat.keys()):
        if item not in ['__header__', '__version__', '__globals__']:
            df_tmp = pd.DataFrame(data=mat[item], columns=['time', 'depth (dBars)', 'temp (C)','salinity (PSU)', 'potential density (kg/m^3 -1000)', 
                                                    'chla (ppb)', 'turbidity (FTU)', 'turb kinetic energy 1 (W/kg)', 'turb kinetic energy 2 (W/kg)'])
            df_tmp['transect'] = names[i]
            df_tmp['profile_num'] = names[i]+'_'+item.split('profile')[-1]
            li.append(df_tmp)

profiles_2021 = pd.concat(li, axis=0, ignore_index=True)
profiles_2021['dt'] = pd.to_datetime('2021-1-1') + pd.to_timedelta(profiles_2021.time, unit='D') - pd.Timedelta(days=1)
profiles_2021 = profiles_2021.set_index('dt')
profiles_2021 = profiles_2021.sort_index(ascending=True)
profiles_2021['datetime'] = pd.to_datetime('2021-1-1') + pd.to_timedelta(profiles_2021.time, unit='D') - pd.Timedelta(days=1)

In [ ]:
vmp_fns = glob.glob('data/VMP/VMP*.mat')
vmp_fns.sort()

names = ['t'+str(i) for i in np.arange(count,len(vmp_fns)+count)]
count += len(vmp_fns)

li = []
for i, fn in enumerate(vmp_fns):
    mat = scipy.io.loadmat(fn)
    for item in list(mat.keys()):
        if item not in ['__header__', '__version__', '__globals__']:
            df_tmp = pd.DataFrame(data=mat[item], columns=['time', 'depth (dBars)', 'temp (C)','salinity (PSU)', 'potential density (kg/m^3 -1000)', 
                                                    'chla (ppb)', 'turbidity (FTU)', 'turb kinetic energy 1 (W/kg)', 'turb kinetic energy 2 (W/kg)'])
            df_tmp['transect'] = names[i]
            df_tmp['profile_num'] = names[i]+'_'+item.split('profile')[-1]
            li.append(df_tmp)

profiles_2022 = pd.concat(li, axis=0, ignore_index=True)
profiles_2022['dt'] = pd.to_datetime('2022-1-1') + pd.to_timedelta(profiles_2022.time, unit='D') - pd.Timedelta(days=1)
profiles_2022 = profiles_2022.set_index('dt')
profiles_2022 = profiles_2022.sort_index(ascending=True)
profiles_2022['datetime'] = pd.to_datetime('2022-1-1') + pd.to_timedelta(profiles_2022.time, unit='D') - pd.Timedelta(days=1)

In [ ]:
profiles = pd.concat([profiles_2020,profiles_2021,profiles_2022])

In [ ]:
from geopy import distance

def get_dist(df, df_start, flip=False):
    distances = []
    # flip allows you to swap the starting side so that it can always do dist from the same side
    if flip == False:
        first_row = df_start.iloc[0]
    else:
        first_row = df_start.iloc[-1]
    for row in df.itertuples(index=False):
        distances.append(distance.distance((row.lat, row.lon),(first_row.lat,first_row.lon)).meters)
    return(distances)

In [ ]:
def profile_locs(df, adcp_df):
    # add in lat and lon
    lats = []
    lons = []
    dists = []
    for i in range(len(df)):
        row = adcp_df.iloc[adcp_df.index.get_loc(pd.to_datetime(df.iloc[i].datetime), method='nearest')]
        lats.append(row.lat)
        lons.append(row.lon)
        dists.append(row.dist)
    df['lat'] = lats
    df['lon'] = lons
    df['dist'] = dists

    # now make it the same for each profile depending on the first row with that profile_num
    profile_nums = df.profile_num.unique()
    profile_num_idx = 0
    
    lats = []
    lons = []
    dists = []
    for row in df.itertuples(index=False):
#         print(profile_num_idx)
        try:
            if row.profile_num == profile_nums[profile_num_idx]:
                top_lat = row.lat
                top_lon = row.lon
                dist = row.dist
                profile_num_idx+=1
        except IndexError:
#             print('index error')
            pass
        lats.append(top_lat)
        lons.append(top_lon)
        dists.append(dist)
        
    df['lat'] = lats
    df['lon'] = lons
    df['dist'] = dists
    return(df)

In [ ]:
from colour import Color
def plot_profiles(df, adcp_df, start_time, end_time, data_variables, plot=False):

#     print('##############################')
# #     print('Transect ' + str(i+1))
#     print('##############################')
    df_subset = df.loc[start_time:end_time]
    
    df_subset = profile_locs(df_subset, adcp_df)
    
    fig, ax = plt.subplots(1,4, figsize=(30,5))
    
    if len(df_subset) == 0:
        print('Did not have profile data.')
        return(ax)
    
    # Original data (e.g. measurements)
    for i, data_var in enumerate(data_variables):
        if plot:
            cmap='RdYlBu_r'
            norm=None
#             fig, ax = plt.subplots(figsize=(17.2,7))
            if data_var == 'temp (C)':
                cmap = cmocean.cm.thermal
#                 vmin=21
#                 vmax=27.5
            elif data_var == 'salinity (PSU)':
                cmap = cmocean.cm.haline
                pass
#                 vmin=34.8
#                 vmax=36.2
            elif data_var == 'potential density (kg/m^3 -1000)':
                cmap = cmocean.cm.dense
#                 vmin=22.5
#                 vmax=23.5
            elif data_var == 'chla (ppb)':
                vmin=0.0
                vmax=0.6
                cmap = cmocean.cm.algae
            elif data_var == 'turbidity (FTU)':
                pass
#                 vmin=1
#                 vmax=1.12
            elif data_var == 'turb kinetic energy 1 (W/kg)':
                vmin=1e-10
                vmax=1e-6
                norm=mat_colors.LogNorm()
            elif data_var == 'turb kinetic energy 2 (W/kg)':
                vmin=1e-10
                vmax=1e-6
                norm=mat_colors.LogNorm()
            elif data_var == 'cluster':
                vmin=0
                vmax=6
                cmap='jet'
#                 norm=mat_colors.LogNorm()
                
#             flip=False
#             if transect in [1,8]:
#                 flip=True
            
#             df_subset_distances = get_dist(df_subset, adcp_df.loc[start_time:end_time], flip=False)

            vmin = np.nanpercentile(df_subset[data_var], 8)
            vmax = np.nanpercentile(df_subset[data_var], 92)
            if data_var == 'ekbackscatter':
                vmin=97.5
                vmax=115
            if data_var == 'cluster':
                vmin=0
                vmax=6
            if data_var == 'salinity (PSU)':
                vmin=35
                vmax=36.5
            if data_var == 'chla (ppb)':
                vmin=0
                vmax=0.6
            sc = ax[i].scatter(df_subset['dist'], df_subset['depth (dBars)'],c=df_subset[data_var],cmap=cmap, s=150, alpha=.7, vmin=vmin,vmax=vmax,)# norm=norm)
            
#             if transect==0:
                
#                 Sep 05 2021 13:15:33 [System UTC, header]
#                 Sep 05 2021 15:21:30 [System UTC, header]
#                 Sep 05 2021 17:14:42 [System UTC, header]
                        
                
#                 if data_var == 'temp (C)':
#                     for t_string in ['Sep 05 2021 13:15:33']:
#                         df_idx = df_subset.index.get_loc(pd.to_datetime('Sep 05 2021 13:15:33'), method='nearest')
#                         ax.scatter(df_subset_distances, df_subset['depth (dBars)'],c=df_subset[data_var]
            
#             sc = ax.scatter(df_subset['datetime'], df_subset['depth (dBars)'],c=df_subset[data_var],cmap=cmap, s=150, alpha=.7, vmin=vmin,vmax=vmax, norm=norm)
            ax[i].set_ylim(-120,0)
#             ax.set_xlim(df_subset['datetime'].max()+timedelta(hours=.1),df_subset['datetime'].min()-timedelta(hours=.1))
            cb = fig.colorbar(sc,ax=ax[i])
#             cb.set_label(data_var)
            ax[i].set_title(data_var)
#             depths = [-5,-10,-20,-30,-60]
        
#             colors = list(Color("black").range_to(Color("blue"),len(depths)))
#             hex_colors = [j.hex for j in colors]
#             # integrate down to z depth
#             ax2 = ax.twinx()
#             for idx,z in enumerate(depths):
#                 # cut it off at depth (dBars) >= z
#                 df_tmp = df_subset[df_subset['depth (dBars)'] >= z] 
#                 df_tmp_grouped = df_tmp.groupby('profile_num').mean()
#                 df_tmp_grouped['datetime'] = pd.to_datetime('2021-1-1') + pd.to_timedelta(df_tmp_grouped.time, unit='D') - pd.Timedelta(days=1)


#                 ax2.plot(df_tmp_grouped['datetime'],df_tmp_grouped[data_var], label=str(z), color=hex_colors[idx])
#                 ax2.set_ylim(vmin,vmax)
#                 ax2.legend(loc='lower left',title='Depth Integrated Values')
            
            
        
            ax[i].set_ylabel('Depth (m')
            ax[i].set_xlabel('Distance (m)')
#             ax.set_xlim(pd.to_datetime(start_time), pd.to_datetime(end_time))
#             ax.set_xlim(pd.to_datetime(end_time),pd.to_datetime(start_time))
        #     ax[1].xaxis.set_major_locator(plt.MaxNLocator(7))
#     fig.autofmt_xdate(rotation=45)
#             ax.xaxis.set_major_locator(plt.MaxNLocator(15))
#             print(data_var[:20]+'.png')
#             fig.savefig('figs/sept_transect_'+str(transect+1)+'_'+data_var[:20]+'.png')
    return(ax)

In [ ]:
from colour import Color
def grab_vmp_data(df, adcp_df, start_time, end_time):

    df_subset = df.loc[start_time:end_time]
    df_subset['DOY'] = df_subset.datetime.dt.day_of_year
    df_subset.sort_index(inplace=True, ascending=True)
    
    df_subset = profile_locs(df_subset, adcp_df)
    z = -4.5
    df_tmp = df_subset[df_subset['depth (dBars)'] >= z] 
    df_tmp_grouped = df_tmp.groupby('profile_num').mean()
    
    df_tmp_grouped.sort_values('dist', inplace=True)
    
    return(df_tmp_grouped['salinity (PSU)'].values, df_tmp_grouped['temp (C)'].values, df_tmp_grouped['chla (ppb)'].values, df_tmp_grouped['turbidity (FTU)'].values, df_tmp_grouped['DOY'].values, df_tmp_grouped['dist'].values)
    

In [ ]:
plt.rcParams.update({'font.size': 16})

In [ ]:
# for idx,fn in enumerate(filenames):
#     if idx not in [39,40,41,42]:
# #     if idx not in range(25,39):
#         continue
#     print(idx)
#     print(fn)
    
# #     try:

#     dist = scipy.io.loadmat(fn+'_distance.mat')['distance']
#     sst = scipy.io.loadmat(fn+'_SST.mat')['SST']
#     time = scipy.io.loadmat(fn+'_time.mat')['time']
#     timestamps = pd.to_datetime(time[0,:]-719529, unit='D')

# #         if idx in [5,28,39,40,41]:
# #             dist = (dist - dist[-1])*-1

#     X = scipy.io.loadmat(fn+'_X.mat')['X']
#     Y = scipy.io.loadmat(fn+'_Y.mat')['Y']
#     vMag = scipy.io.loadmat(fn+'_vmag.mat')['C']

#     vDir = scipy.io.loadmat(fn+'_vdir.mat')['C']

#     echo = scipy.io.loadmat(fn+'_echo.mat')['C']
#     Xecho = scipy.io.loadmat(fn+'_Xecho.mat')['X']
#     Yecho = scipy.io.loadmat(fn+'_Yecho.mat')['Y']
#     # correcting for the smaller bin size
#     Yecho = (Yecho*0.502)-1

#     lat = scipy.io.loadmat(fn+'_lat.mat')['lat']
#     long = scipy.io.loadmat(fn+'_long.mat')['long']
    
#     adcp_df = pd.DataFrame({'dt':timestamps,
#                     'datetime':timestamps,
#                    'lat': lat[:,0],
#                    'lon':long[:,0],
#                            'dist':dist[:,0]})

#     adcp_df = adcp_df.set_index('dt')
#     adcp_df = adcp_df.sort_index(ascending=True)

#     plot_profiles(profiles, adcp_df, timestamps[0],timestamps[-1], ['temp (C)', 'salinity (PSU)', 'chla (ppb)'], plot=True)

#     fig, ax = plt.subplots(3,1, figsize=(10,12))
#     vMag_reshape = vMag.reshape(-1,69).T
#     vDir_reshape = vDir.reshape(-1,69).T

# #         if idx in [5,28,39,40,41]:
# #             vMag_reshape = np.fliplr(vMag_reshape)
# #             vDir_reshape = np.fliplr(vDir_reshape)

#     x, y = np.meshgrid(X.reshape(-1,69)[:,0],Y[:69,0])    
#     im = ax[0].pcolormesh(x,y,vMag_reshape,shading='gouraud', vmin=10,vmax=140)
#     fig.colorbar(im,ax=ax[0])
#     ax[0].set_title("Current Magnitude (cm/s)")

#     im = ax[1].pcolormesh(x,y,vDir_reshape,shading='gouraud', vmin=0,vmax=360, cmap='hsv')
#     fig.colorbar(im,ax=ax[1])
#     ax[1].set_title("Current Direction (Deg from North)")

#     vgradient = []

#     N=180
#     current_speed = np.mean(vMag_reshape[0:4,:],axis=0)
#     current_speed_smooth = pd.Series(current_speed).rolling(window=N).mean().iloc[N-1:].values
#     vgradient = []

#     step = 1

#     for i in range(len(current_speed)-N):
#         du = current_speed_smooth[i] - current_speed_smooth[i+step]
#         dx = dist[i] - dist[i+step]
#         current_grad = du/dx
#         vgradient.append(current_grad)

#     dudx = scipy.signal.savgol_filter(current_speed_smooth, window_length=11, polyorder=2, deriv=1)

#     ax_twin = ax[0].twinx()
# #         ax_twin.plot(dist[15:-15-N], vgradient_smooth[15:-16], c='k', alpha=0.5)
#     ax_twin.plot(dist[:-N+1], dudx, c='grey', ls='--')
#     ax_twin.set_ylim(-0.6,0.6)

#     ax_twin2 = ax[0].twinx()
#     ax_twin2.plot(dist[:-N],current_speed_smooth[:-1],c='black')
#     ax_twin2.set_ylim(0,140)

#     ax_twin3 = ax[0].twinx()
#     ax_twin3.plot(dist,sst, color='red')
#     ax_twin3.set_ylim(25,30)
#     # taking off the first 100 points because those are often noisy and never the front
#     front_location = np.argmax(np.abs(dudx[100:-30]))+100

#     echo_reshape = echo.reshape(-1,127).T
# #         vDir_reshape = vDir.reshape(-1,69).T

# #         if idx in [5,28,39,40,41]:
# #             echo_reshape = np.fliplr(echo_reshape)

#     x, y = np.meshgrid(Xecho.reshape(-1,127)[:,0],Yecho[:127,0])    

#     im = ax[2].pcolormesh(x,y,echo_reshape,shading='gouraud', vmin=96,vmax=115, cmap='jet')
#     fig.colorbar(im,ax=ax[2])
#     ax[2].set_title("Echosounder Volume Backscatter (sV)")

#     ax_twin.yaxis.label.set_color('grey')
#     ax_twin.spines['right'].set_position(('outward', 90))

#     ax_twin2.yaxis.label.set_color('black')
#     ax_twin2.spines['right'].set_position(('outward', 180))

#     ax_twin3.yaxis.label.set_color('red')
#     ax_twin3.spines['right'].set_position(('outward', 260))

#     ax_twin.set_ylabel('current gradient')
#     ax_twin2.set_ylabel('top 10m current speed (cm/s)')
#     ax_twin3.set_ylabel('SST (C)')


#     if idx == 42:
#         for i in range(3):
#             ax[i].set_xlim(dist.max(),0)
#     else:
#         for i in range(3):
#             ax[i].set_xlim(0,dist.max())
#     for i in range(3):
#         ax[i].axvline(dist[front_location],color='k', ls='--')
#         ax[i].set_ylim(-70,-2)
#         ax[i].set_ylabel('Depth (m)')
#         ax[i].set_xlabel('Dist from Start (m)')
#         ax[i].axvline(dist[front_location],color='k', ls='--')

#     fig.tight_layout()



# #         fig.savefig('mag_dir_echo'+str(idx)+'.png',dpi=400)
#     plt.show()

#     print('--------------------------------------')

# #     except Exception as e: 
# #         print(e)
    
#     break


In [ ]:
sst_seasonal = pd.read_csv('sst_gs_coastal.csv',names=['datetime', 'coastal_sst', 'gs_sst'], skiprows=1)
chla_seasonal = pd.read_csv('chla_gs_coastal.csv',names=['datetime', 'coastal_chla', 'gs_chla'], skiprows=1)

sst_seasonal.datetime = pd.to_datetime(sst_seasonal.datetime)
sst_seasonal = sst_seasonal.set_index('datetime')

chla_seasonal.datetime = pd.to_datetime(chla_seasonal.datetime)
chla_seasonal = chla_seasonal.set_index('datetime')

In [ ]:
fig,ax = plt.subplots(figsize=(8,5))

dates_x_sst = np.arange(1,366,1)

####################

sst_mean = sst_seasonal.groupby([sst_seasonal.index.month, sst_seasonal.index.day]).mean()
sst_std = sst_seasonal.groupby([sst_seasonal.index.month, sst_seasonal.index.day]).std()

ax.plot(dates_x_sst,sst_mean.gs_sst, c='blue', label='Gulf Stream')
ax.plot(dates_x_sst,sst_mean.gs_sst+sst_std.gs_sst, c='blue', alpha=0.2)
ax.plot(dates_x_sst,sst_mean.gs_sst-sst_std.gs_sst, c='blue', alpha=0.2)

###################

ax.plot(dates_x_sst,sst_mean.coastal_sst, c='green', label='Coastal')
ax.plot(dates_x_sst,sst_mean.coastal_sst+sst_std.coastal_sst, c='green', alpha=0.2)
ax.plot(dates_x_sst,sst_mean.coastal_sst-sst_std.coastal_sst, c='green', alpha=0.2)

################3

# ax.legend(loc='lower right')

ax.set_ylabel('SST (C)')
ax.set_xlabel('Day of Year')

plt.savefig('gs_coastal_sst.png',dpi=300)

In [ ]:
fig,ax = plt.subplots(figsize=(8,5))

chla_mean = chla_seasonal.groupby([chla_seasonal.index.month, chla_seasonal.index.day]).mean()
chla_std = chla_seasonal.groupby([chla_seasonal.index.month, chla_seasonal.index.day]).std()

str_dts = []
for i in range(len(chla_mean.index)):
    str_dts.append(str(chla_mean.index[i][1])+','+str(chla_mean.index[i][0]))
    
day_of_year_chla = [datetime.strptime(dt,"%d,%m").timetuple().tm_yday for dt in str_dts]

dates_x_chla = day_of_year_chla

####################

ax.plot(dates_x_chla,chla_mean.gs_chla, c='blue', label='Gulf Stream')
ax.plot(dates_x_chla,chla_mean.gs_chla+chla_std.gs_chla, c='blue', alpha=0.2)
ax.plot(dates_x_chla,chla_mean.gs_chla-chla_std.gs_chla, c='blue', alpha=0.2)

###################

ax.plot(dates_x_chla,chla_mean.coastal_chla, c='green', label='Coastal')
ax.plot(dates_x_chla,chla_mean.coastal_chla+chla_std.coastal_chla, c='green', alpha=0.2)
ax.plot(dates_x_chla,chla_mean.coastal_chla-chla_std.coastal_chla, c='green', alpha=0.2)

################3

ax.set_ylim(0,2.75)

# ax.legend()

ax.set_ylabel('Chla (mg/m3)')
ax.set_xlabel('Day of Year')
plt.savefig('gs_coastal_chla.png',dpi=300)

Then we'll run the subpixel_contours() function which takes in an xarray DataArray and related geospatial metadata and returns a geodataframe of the contours at each time step.

To do this we need to define an affine transformation for the SSH data since it doesn't exist in the dataset currently. Learn more [here](https://www.perrygeo.com/python-affine-transforms.html). The basic format of an affine transformation is (a, b, c, d, e, f):

    a = width of a pixel
    b = row rotation (typically zero)
    c = x-coordinate of the upper-left corner of the upper-left pixel
    d = column rotation (typically zero)
    e = height of a pixel (typically negative)
    f = y-coordinate of the of the upper-left corner of the upper-left pixel

In [ ]:
from affine import Affine
from dea_spatial import subpixel_contours

# ssh_ds = xr.open_zarr('data/aviso.zarr')
ssh_ds = xr.open_dataset('data/dataset-duacs-nrt-global-merged-allsat-phy-l4_1660503138026.nc')
ssh_ds.adt.attrs["units"] = 'meters'

ssh_affine = Affine(0.25, 0.0, -81.875-.25/2, 0.0, 0.25, 26.125-.25/2)
# ssh_affine = Affine(0.25, 0.0, -82.12-.25/2, 0.0, 0.25, 43.62-.25/2)

# this function needs all the data in memory so load it in
ssh_ds.adt.load()

front_gdf_median = subpixel_contours(ssh_ds.adt.median(dim='time', skipna=True), [0.25], crs='EPSG:4326', min_vertices=50, affine=ssh_affine, verbose=True)

In [ ]:
front_gdf = subpixel_contours(ssh_ds.adt, [0.25], crs='EPSG:4326', min_vertices=50, affine=ssh_affine, verbose=True)

In [ ]:
front_gdf['dt'] = pd.to_datetime(front_gdf.time)
front_gdf = front_gdf.set_index('dt')

In [ ]:
front_gdf

In [ ]:
fig,ax = plt.subplots()

front_gdf.plot(ax=ax)
front_gdf_median.plot(ax=ax,color='black')

### Define the salinity and temp models

In [ ]:
dist_list = []
front_location_list = []
sst_list = []
current_list = []
echo_list = []
interp_sst_list = []
interp_current_list = []
interp_echo_list = []

sal_front_list = []

vmp_sal_list = []
vmp_temp_list = []
vmp_chla_list = []
vmp_dt_list = []

for idx,fn in enumerate(filenames):
    if idx in [0,1,2,3]:
        continue
    try:
        ##########
        
        dist = scipy.io.loadmat(fn+'_distance.mat')['distance']
        sst = scipy.io.loadmat(fn+'_SST.mat')['SST']
        time = scipy.io.loadmat(fn+'_time.mat')['time']
        timestamps = pd.to_datetime(time[0,:]-719529, unit='D')
  
        lat = scipy.io.loadmat(fn+'_lat.mat')['lat']
        long = scipy.io.loadmat(fn+'_long.mat')['long']
        
        # the distances are crazy wrong sometimes so re-calcuating the distance
        coords_1 = (lat[0], long[0])
        distances = []
        for i in range(len(lat)):
            coords_2 = (lat[i], long[i])
            distances.append(distance.geodesic(coords_1, coords_2).meters)
        dist = np.array(distances)       

        # checking if the current point is within 500 of one that was good, assuming the first point is good
        # then if not setting it to nan and interpolating it out
        last_good_idx = 0
        bad_idxs = []
        for di,d in enumerate(dist):
            if dist[di] - dist[last_good_idx] < 500:
                last_good_idx = di
            else:
                bad_idxs.append(di)
        dist[bad_idxs] = np.nan
        dist = pd.Series(dist).interpolate().values

        dist = dist.reshape(-1,1)
        
        adcp_df = pd.DataFrame({'dt':timestamps,
                    'datetime':timestamps,
                    'sst':sst[:,0],
                    'lat': lat[:,0],
                    'lon':long[:,0],
                    'dist':dist[:,0]})

        adcp_df = adcp_df.set_index('dt')
        adcp_df = adcp_df.sort_index(ascending=True)
        
        
        ##########
        
        sal_vmp, temp_vmp, chla_vmp, turb_vmp, dt_vmp, dist_vmp = grab_vmp_data(profiles, adcp_df, timestamps[0],timestamps[-1])
            
        vmp_sal_list.append(sal_vmp)
        vmp_temp_list.append(temp_vmp)
        vmp_chla_list.append(chla_vmp)
        vmp_dt_list.append(dt_vmp)
        

    except Exception as e: 
        print(e)



how many transects do we have with VMP data

In [ ]:
1

In [ ]:
count = 0
for trans in vmp_sal_list:
    if len(trans) > 0:
        count += 1
count

In [ ]:
import gsw

In [ ]:
plt.hist(np.concatenate(vmp_sal_list), bins=80)
plt.show()

## calculate the seasonal dependence of salinity

In [ ]:
max_sal_list = []
for i in vmp_sal_list:
    if len(i) == 0:
        continue
    max_sal_list.append(np.percentile(i, 95))

doy_list = []
for i in vmp_dt_list:
    if len(i) == 0:
        continue
    doy_list.append(np.max(i))

In [ ]:
# plt.scatter(abs(np.array(doy_list)-225),max_sal_list)
plt.scatter(doy_list,max_sal_list)

In [ ]:
import numpy as np
from scipy.optimize import leastsq
import pylab as plt

t = np.array(doy_list)
data = np.array(max_sal_list)

guess_mean = np.mean(data)
guess_std = 3*np.std(data)/(2**0.5)/(2**0.5)
guess_phase = 0
guess_freq = 1/60
guess_amp = .3

# we'll use this to plot our first estimate. This might already be good enough for you
data_first_guess = guess_amp*np.sin(guess_freq*t+guess_phase) + guess_mean

# Define the function to optimize, in this case, we want to minimize the difference
# between the actual data and our "guessed" parameters
optimize_func = lambda x: x[0]*np.sin(x[1]*t+x[2]) + x[3] - data
est_amp, est_freq, est_phase, est_mean = leastsq(optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean])[0]

# recreate the fitted curve using the optimized parameters
data_fit = est_amp*np.sin(est_freq*t+est_phase) + est_mean

# recreate the fitted curve using the optimized parameters

fine_t = np.arange(0,max(t),0.1)
data_fit=est_amp*np.sin(est_freq*fine_t+est_phase)+est_mean

plt.plot(t, data, '.')
plt.scatter(t, data_first_guess, label='first guess', color='red')
plt.plot(fine_t, data_fit, label='after fitting')
plt.legend()
plt.show()

In [ ]:
est_amp, 1/est_freq, est_phase, est_mean

## calculate the seasonal dependence of temperature

In [ ]:
max_temp_list = []
for i in vmp_temp_list:
    if len(i) == 0:
        continue
    max_temp_list.append(np.percentile(i, 95))

In [ ]:
import numpy as np
from scipy.optimize import leastsq
import pylab as plt

t = dates_x_sst
data = sst_mean.gs_sst-273

t = np.array(doy_list)
data = np.array(max_temp_list)


guess_mean = np.mean(data)
guess_std = 3*np.std(data)/(2**0.5)/(2**0.5)
guess_phase = 230
guess_freq = 1/60
guess_amp = 5

# we'll use this to plot our first estimate. This might already be good enough for you
data_first_guess = guess_amp*np.sin(guess_freq*t+guess_phase) + guess_mean

# Define the function to optimize, in this case, we want to minimize the difference
# between the actual data and our "guessed" parameters
optimize_func = lambda x: x[0]*np.sin(x[1]*t+x[2]) + x[3] - data
sst_est_amp, sst_est_freq, sst_est_phase, sst_est_mean = leastsq(optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean])[0]

# recreate the fitted curve using the optimized parameters
data_fit = sst_est_amp*np.sin(sst_est_freq*t+sst_est_phase) + sst_est_mean

# recreate the fitted curve using the optimized parameters

fine_t = np.arange(0,max(t),0.1)
data_fit=sst_est_amp*np.sin(sst_est_freq*fine_t+sst_est_phase)+sst_est_mean

plt.plot(t, data, '.')
plt.scatter(t, data_first_guess, label='first guess')
plt.plot(fine_t, data_fit, label='after fitting')
plt.legend()
plt.show()

In [ ]:
sst_est_amp, 1/sst_est_freq, sst_est_phase, sst_est_mean

almost the exact same when fitting with the satellite data

In [ ]:
sst_est_amp, 1/sst_est_freq, sst_est_phase, sst_est_mean

In [ ]:
sst_est_phase

In [ ]:
doy = np.arange(0,365,1)
def provide_gs_temp_threshold(doy,buffer=0.0):
    temp_est = sst_est_amp*np.sin(sst_est_freq*doy+sst_est_phase) + sst_est_mean
    return(temp_est-buffer)

temp_est = provide_gs_temp_threshold(doy)

plt.scatter(doy,temp_est)

In [ ]:
doy = np.arange(0,365,1)
def provide_gs_sal_threshold(doy,buffer=0.1):
    sal_est = est_amp*np.sin(est_freq*doy+est_phase) + est_mean
    return(sal_est-buffer)

sal_est = provide_gs_sal_threshold(doy,buffer=0)
temp_est = provide_gs_temp_threshold(doy,buffer=0)


fig, ax = plt.subplots(figsize=(10,7))
ax.plot(doy,sal_est, c='blue', label='salinity')
ax.set_ylabel('Salinity (PSU)')
ax2 = ax.twinx()
ax2.plot(doy,temp_est, c='red', label='temp')
ax2.set_ylabel('Temp (C)')
ax.axvline(42, c='k', ls='--')
ax.axvline(230,c='k', ls='--')

ax.scatter(np.array(doy_list),np.array(max_sal_list), c='blue')
ax2.scatter(np.array(doy_list),np.array(max_temp_list), c='red')

ax.set_ylim(35.4,36.6)

ax.set_xlabel('Day of Year')

ax.legend(loc='lower right')
ax2.legend(loc='upper right')

ax.set_title('Modeled Gulf Stream S and T')

annual_sal_mean = np.mean(sal_est)
annual_temp_mean = np.mean(temp_est)

# fig.savefig('sine_model_s_t.png',dpi=300)

This defines some parameters for the echosounder backscatter calculations

In [ ]:
ed_gs_depth = 8*2
shelf_depth = 8*2

gulf_stream_ek = []
eddy_ek = []
shelf_ek = []

In [ ]:
echo_reshape.shape

In [ ]:
dist.shape

In [ ]:
echo_reshape.shape

In [ ]:
profiles['ekbackscatter'] = np.nan
profiles['current_mag'] = np.nan

for idx,fn in enumerate(filenames):
    if idx in [0,1,2,3]:
        continue
    try:
        ##########
#     else:
        dist = scipy.io.loadmat(fn+'_distance.mat')['distance']
        sst = scipy.io.loadmat(fn+'_SST.mat')['SST']
        time = scipy.io.loadmat(fn+'_time.mat')['time']
        timestamps = pd.to_datetime(time[0,:]-719529, unit='D')
       
        
#         if idx in [5,28,39,40,41]:
#             dist = (dist - dist[-1])*-1
        
        X = scipy.io.loadmat(fn+'_X.mat')['X']
        Y = scipy.io.loadmat(fn+'_Y.mat')['Y']
        vMag = scipy.io.loadmat(fn+'_vmag.mat')['C']

        vDir = scipy.io.loadmat(fn+'_vdir.mat')['C']
        
        vShear = scipy.io.loadmat(fn+'_vshear.mat')['C']
        
        echo = scipy.io.loadmat(fn+'_echo.mat')['C']
        Xecho = scipy.io.loadmat(fn+'_Xecho.mat')['X']
        Yecho = scipy.io.loadmat(fn+'_Yecho.mat')['Y']
        
        # correcting for the smaller bin size
        Yecho = (Yecho*0.502)-1
        
        echo_reshape = echo.reshape(-1,127).T
        vMag_reshape = vMag.reshape(-1,69).T
        
        lat = scipy.io.loadmat(fn+'_lat.mat')['lat']
        long = scipy.io.loadmat(fn+'_long.mat')['long']
        
        # the distances are crazy wrong sometimes so re-calcuating the distance
        coords_1 = (lat[0], long[0])
        distances = []
        for i in range(len(lat)):
            coords_2 = (lat[i], long[i])
            distances.append(distance.geodesic(coords_1, coords_2).meters)
        dist = np.array(distances)       
#         bogus_dist_locations = dist > 35000        
#         dist[dist > 25000] = np.nan
#         dist = pd.Series(dist).interpolate().values
        # checking if the current point is within 500 of one that was good, assuming the first point is good
        # then if not setting it to nan and interpolating it out
        last_good_idx = 0
        bad_idxs = []
        for di,d in enumerate(dist):
            if dist[di] - dist[last_good_idx] < 500:
                last_good_idx = di
            else:
                bad_idxs.append(di)
        dist[bad_idxs] = np.nan
        dist = pd.Series(dist).interpolate().values

        dist = dist.reshape(-1,1)
        
        adcp_df = pd.DataFrame({'dt':timestamps,
                    'datetime':timestamps,
                    'sst':sst[:,0],
                    'lat': lat[:,0],
                    'lon':long[:,0],
                    'dist':dist[:,0]})

        adcp_df = adcp_df.set_index('dt')
        adcp_df = adcp_df.sort_index(ascending=True)
        
        
        # find the adcp data with the closest location to each profile in distance space 
        # and then grab the echosounder data from that
        
        # the logic could be
        # grab the closest ek sounding in time with the profile data
        # then use that distance to average +/- the last 10 EK soundings
        # interpolate that into the profile depth and don't extrapolate
        # do this for each transect via just having a time tolerance of ~30 seconds
        profiles_subset = profiles.loc[timestamps[0]:timestamps[-1]]
        profiles_subset = profile_locs(profiles_subset, adcp_df)
        
        profile_nums = profiles_subset.profile_num.unique()
        profile_num_idx = 0

        for row in profiles_subset.itertuples(index=False):
            try:
                if row.profile_num == profile_nums[profile_num_idx]:
#                     print(np.abs(dist - row.dist).min())
                    dist_arg = np.abs(dist - row.dist).argmin()
                    # we don't want the matchs to be very far away and better to keep as nans if so
                    if np.abs(dist - row.dist).min() > 5:
                        print('greater than 5m')
                        continue
                    ek_for_profile = np.interp(abs(profiles_subset[profiles_subset.profile_num == profile_nums[profile_num_idx]]['depth (dBars)'].values), abs(Yecho[:127,0]), np.nanmedian(echo_reshape[:,dist_arg-20:dist_arg+20],axis=1),left=np.nan, right=np.nan)
                    mag_for_profile = np.interp(abs(profiles_subset[profiles_subset.profile_num == profile_nums[profile_num_idx]]['depth (dBars)'].values), abs(Y[:69,0]), np.nanmedian(vMag_reshape[:,dist_arg-20:dist_arg+20],axis=1),left=np.nan, right=np.nan)
         
                    
                    m = profiles_subset.profile_num == profile_nums[profile_num_idx]
                    
                    profiles_subset.loc[m,'ekbackscatter'] = ek_for_profile
                    profiles_subset.loc[m,'current_mag'] = mag_for_profile
                    
                    profile_num_idx+=1

            except IndexError:
        #         print('index error')
                pass

        profiles.loc[timestamps[0]:timestamps[-1],'ekbackscatter'] = profiles_subset.ekbackscatter 
        profiles.loc[timestamps[0]:timestamps[-1],'current_mag'] = profiles_subset.current_mag 
#         break
        
    except Exception as e: 
        print(e)



In [ ]:
ek_for_profile = np.interp(abs(profiles_subset[profiles_subset.profile_num == profile_nums[profile_num_idx-1]]['depth (dBars)'].values), abs(Yecho[:127,0]), np.nanmedian(echo_reshape[:,dist_arg-20:dist_arg+20],axis=1),left=np.nan, right=np.nan)

In [ ]:
plt.plot(ek_for_profile,profiles_subset[profiles_subset.profile_num == profile_nums[profile_num_idx-1]]['depth (dBars)'].values)
plt.plot(np.nanmedian(echo_reshape[:,dist_arg-20:dist_arg+20],axis=1),Yecho[:127,0])

In [ ]:
vmag_for_profile = np.interp(abs(profiles_subset[profiles_subset.profile_num == profile_nums[profile_num_idx-1]]['depth (dBars)'].values), abs(Y[:69,0]), np.nanmedian(vMag_reshape[:,dist_arg-20:dist_arg+20],axis=1),left=np.nan, right=np.nan)

In [ ]:
vmag_for_profile

In [ ]:
plt.plot(vmag_for_profile,profiles_subset[profiles_subset.profile_num == profile_nums[profile_num_idx-1]]['depth (dBars)'].values)
plt.plot(np.nanmedian(vMag_reshape[:,dist_arg-20:dist_arg+20],axis=1),Y[:69,0])

In [ ]:
# Figure out boudaries (mins and maxs)
smin = 32-.7
smax = 37+.7
tmin = 8-.7 
tmax = 30+.7

# Calculate how many gridcells we need in the x and y dimensions
xdim = int(round((smax-smin)/0.1+1,0))
ydim = int(round((tmax-tmin)+1,0))
 
# Create empty grid of zeros
dens = np.zeros((ydim,xdim))
 
# Create temp and salt vectors of appropiate dimensions
ti = np.linspace(1,ydim-1,ydim)+tmin
si = np.linspace(1,xdim-1,xdim)*0.1+smin
 
# Loop to fill in grid with densities
for j in range(0,int(ydim)):
    for i in range(0, int(xdim)):
        dens[j,i]=gsw.rho(si[i],ti[j],0)

# Substract 1000 to convert to sigma-t
dens = dens - 1000

In [ ]:
import cmocean

In [ ]:
# profiles['cluster'] = kmeans.labels_

profs_turb_sub['cluster'] = kmeans.labels_

In [ ]:
df_tmp_grouped.dist,df_tmp_grouped['potential density (kg/m^3 -1000)']

In [ ]:
front_transects = []
front_locations = []
time_starts = []
time_stops = []

for idx,fn in enumerate(filenames):
#     if idx not in tentative_good_idxs:
# #         continue
    if idx in [0,1,2,3]:
        continue
#     if idx not in [16, 17, 30, 36,43, 49,52,64]:
#         continue
#     if idx != 9:
#         continue
#     if idx not in [38,58]:
#         continue

    print(idx)
    print(fn)
    print('---')
    print(datetimes[idx].strftime("%d. %B %Y %I:%M%p UTC"))
    
    try:

        dist = scipy.io.loadmat(fn+'_distance.mat')['distance']
        sst = scipy.io.loadmat(fn+'_SST.mat')['SST']
        time = scipy.io.loadmat(fn+'_time.mat')['time']
        timestamps = pd.to_datetime(time[0,:]-719529, unit='D')
    
       
        
#         if idx in [5,28,39,40,41]:
#             dist = (dist - dist[-1])*-1
        
        X = scipy.io.loadmat(fn+'_X.mat')['X']
        Y = scipy.io.loadmat(fn+'_Y.mat')['Y']
        vMag = scipy.io.loadmat(fn+'_vmag.mat')['C']

        vDir = scipy.io.loadmat(fn+'_vdir.mat')['C']
        
        vShear = scipy.io.loadmat(fn+'_vshear.mat')['C']
        
        echo = scipy.io.loadmat(fn+'_echo.mat')['C']
        Xecho = scipy.io.loadmat(fn+'_Xecho.mat')['X']
        Yecho = scipy.io.loadmat(fn+'_Yecho.mat')['Y']
        
        # correcting for the smaller bin size
        Yecho = (Yecho*0.502)-1
        
        lat = scipy.io.loadmat(fn+'_lat.mat')['lat']
        long = scipy.io.loadmat(fn+'_long.mat')['long']
        
        # the distances are crazy wrong sometimes so re-calcuating the distance
        coords_1 = (lat[0], long[0])
        distances = []
        for i in range(len(lat)):
            coords_2 = (lat[i], long[i])
            distances.append(distance.geodesic(coords_1, coords_2).meters)
        dist = np.array(distances)       
#         bogus_dist_locations = dist > 35000        
#         dist[dist > 25000] = np.nan
#         dist = pd.Series(dist).interpolate().values
        # checking if the current point is within 500 of one that was good, assuming the first point is good
        # then if not setting it to nan and interpolating it out
        last_good_idx = 0
        bad_idxs = []
        for di,d in enumerate(dist):
            if dist[di] - dist[last_good_idx] < 500:
                last_good_idx = di
            else:
                bad_idxs.append(di)
        dist[bad_idxs] = np.nan
        dist = pd.Series(dist).interpolate().values

        dist = dist.reshape(-1,1)
        
        adcp_df = pd.DataFrame({'dt':timestamps,
                    'datetime':timestamps,
                    'sst':sst[:,0],
                    'lat': lat[:,0],
                    'lon':long[:,0],
                    'dist':dist[:,0]})

        adcp_df = adcp_df.set_index('dt')
        adcp_df = adcp_df.sort_index(ascending=True)
        
        adcp_gdf = gpd.GeoDataFrame(
            adcp_df, geometry=gpd.points_from_xy(adcp_df.lon, adcp_df.lat))
        
        ##################
        # calculate the front
        
        sal_vmp, temp_vmp, chla_vmp, turb_vmp, dt_vmp, dist_vmp  = grab_vmp_data(profiles, adcp_df, timestamps[0],timestamps[-1])
        
        # find the location using the salinity gs threshold I made and then find where the change happens
        sal_est = provide_gs_sal_threshold(timestamps[0].timetuple().tm_yday, buffer=0.6)
        gs_water = sal_vmp > sal_est
        idx_of_sal_front = np.where(gs_water[:-1] != gs_water[1:])[0]
        sal_front_location = (dist_vmp[idx_of_sal_front]+dist_vmp[idx_of_sal_front+1])/2
#         if not sal_front_location.size > 0:
#             continue
            
        front_transects.append(idx)
        front_locations.append(sal_front_location)
        time_starts.append(timestamps[0])
        time_stops.append(timestamps[-1])
#         if True:
#             continue
        
        
        ###################
        # GOES STUFF
        
        # find the time step with the most data
        da_subset = goes_ds.sel(time=slice((adcp_df.datetime.min()-timedelta(hours=24)), (adcp_df.datetime.max()+timedelta(hours=24)))).sea_surface_temperature
        max_coverage_index = da_subset.count(dim=['latitude','longitude']).argmax()

        ###################
        
        ########################################
        
        f = plt.figure(figsize=(30,5))
        ax = f.add_subplot(141, projection=crs.PlateCarree())
        ax2 = f.add_subplot(142, projection=crs.PlateCarree())
        ax3 = f.add_subplot(143)
        
        
        ax4 = f.add_subplot(144)
        
        ax.set_title('Location of Transect and SSH Front (0.25m)')
        ax2.set_title('Location Zoom')
        ax4.set_title('Annual SST Cycle')
        
#         da_subset[max_coverage_index].plot(ax=ax, cmap='inferno')
        goes_ds.sel(time=slice((adcp_df.datetime.min()-timedelta(hours=18)), (adcp_df.datetime.max()+timedelta(hours=18)))).sea_surface_temperature.mean(dim='time').plot(ax=ax, cmap=cmocean.cm.thermal)

        ax.coastlines(resolution='10m')
        adcp_gdf.plot(ax=ax, markersize=2, color='green', alpha=1)
#         front_gdf.iloc[front_gdf.index.get_loc(timestamps[0],method='nearest')-1:front_gdf.index.get_loc(timestamps[0],method='nearest')].plot(ax=ax, label='Todays SSH Front')
#         front_gdf_median.plot(ax=ax, color='black',alpha=0.3, label='Average SSH Front')
        
#         ax.legend()
                                                    
#         ax.set_ylim(35.13000031,36.40999985)
#         ax.set_xlim(-75.99000549,-74.11000061)
        # ax.set_ylim(35.43000031,36.40999985)
        # ax.set_xlim(-74.99000549,-74.11000061)
        
        
        adcp_gdf.plot(ax=ax2, markersize=2, column='sst', alpha=1,  cmap='inferno', legend=True)
#         da_subset[max_coverage_index].plot(ax=ax2, vmin=24.5, vmax=30, cmap='inferno')
        
        
        ##################
        
#         ax3.scatter(sal_vmp,temp_vmp, c=dist_vmp)
        ax3.scatter(dist_vmp,temp_vmp, color='red', label='Temp')
        for sal_front in sal_front_location:
            ax3.axvline(sal_front, c='k', ls='--')
        ax3_twin = ax3.twinx()
        ax3_twin.scatter(dist_vmp,sal_vmp, color='blue', label='Salinity')
        ax3_twin.set_ylim(30,36.5)
            
        temp_est = provide_gs_temp_threshold(timestamps[0].timetuple().tm_yday)
        ax3.axhline(temp_est, ls='--', c='red', label='T Gulf Stream')
        ax3_twin.axhline(sal_est, ls='--', c='blue', label='S Gulf Stream')
        
        ax3_twin.legend(loc='lower left')
        ax3.legend(loc='lower right')
    
        ax3.plot(dist,sst, color='red')
        ax3.set_title("T and S (from 0 to 10 m)")
        ax3.set_ylabel('Temp (C)')
        ax3.set_xlabel('Dist from Start (m)')
        ax3.set_ylim(15,30)
#         ax3.axvline(dist[front_location],color='grey', ls='--')

        if idx not in [5,28,39,40,41]:
            ax3.set_xlim(dist.max(),0)    
        else:
            ax3.set_xlim(0,dist.max())
        

#         ax2.scatter(long[front_location],lat[front_location],s=100,c='grey')

        ####################
        # this plots the annual chla data 

#         ax3.plot(dates_x_chla,chla_mean.gs_chla, c='blue', label='Gulf Stream')
#         ax3.plot(dates_x_chla,chla_mean.gs_chla+chla_std.gs_chla, c='blue', alpha=0.2)
#         ax3.plot(dates_x_chla,chla_mean.gs_chla-chla_std.gs_chla, c='blue', alpha=0.2)

#         ax3.plot(dates_x_chla,chla_mean.coastal_chla, c='green', label='Coastal')
#         ax3.plot(dates_x_chla,chla_mean.coastal_chla+chla_std.coastal_chla, c='green', alpha=0.2)
#         ax3.plot(dates_x_chla,chla_mean.coastal_chla-chla_std.coastal_chla, c='green', alpha=0.2)

#         ax3.set_ylim(0,2.75)
        
#         ax3.axvline(timestamps[0].timetuple().tm_yday, c='black')
#         ax3.set_xlim(0,365)
        
#         ax3.legend()

#         ax3.set_ylabel('Chla (mg/m3)')
#         ax3.set_xlabel('Day of Year')
        
        #################
        
#         ax4.scatter([dt_element.timetuple().tm_yday for dt_element in datetimes_subset],np.nanmax(interp_sst_list,axis=1)+273, color='blue')
#         ax4.scatter([dt_element.timetuple().tm_yday for dt_element in datetimes_subset],np.nanmin(interp_sst_list,axis=1)+273, color='green')
        

        ax4.plot(dates_x_sst,sst_mean.gs_sst, c='blue', label='Gulf Stream')
        ax4.plot(dates_x_sst,sst_mean.gs_sst+sst_std.gs_sst, c='blue', alpha=0.2)
        ax4.plot(dates_x_sst,sst_mean.gs_sst-sst_std.gs_sst, c='blue', alpha=0.2)

        ###################

        ax4.plot(dates_x_sst,sst_mean.coastal_sst, c='green', label='Coastal')
        ax4.plot(dates_x_sst,sst_mean.coastal_sst+sst_std.coastal_sst, c='green', alpha=0.2)
        ax4.plot(dates_x_sst,sst_mean.coastal_sst-sst_std.coastal_sst, c='green', alpha=0.2)

        ################3

        ax4.legend(loc='lower right')

        ax4.set_ylabel('SST (K)')
        ax4.set_xlabel('Day of Year')
        
        ax4.axvline(timestamps[0].timetuple().tm_yday, c='black', label='Current Day')
        ax4.set_xlim(0,365)
        ax4.legend()
        
        
        ########################################
        
        ax = plot_profiles(profiles, adcp_df, timestamps[0],timestamps[-1], ['potential density (kg/m^3 -1000)','temp (C)', 'salinity (PSU)', 'chla (ppb)'], plot=True)
        if idx not in [5,28,39,40,41]:
            for i in range(4):
                ax[i].set_xlim(dist.max(),0)    
        else:
            for i in range(4):
                ax[i].set_xlim(0,dist.max())
        for i in range(4):
            ax[i].set_ylim(-100,-1)
#             for sal_front in sal_front_location:
#                 ax[i].axvline(sal_front, c='k', ls='--')
#         plt.savefig('profiles'+str(idx)+'.png',dpi=300)
        plt.show()
    
    
    
        ########################################
        
        
        fig, ax = plt.subplots(1,1, figsize=(8,5))
        
       
        profiles_subset = profiles.loc[timestamps[0]:timestamps[-1]]
    
        profiles_subset = profile_locs(profiles_subset, adcp_df)
        
#         integrate down to z depth
        depths = [-5,-10,-20,-40,-60]
        
        colors = list(Color("black").range_to(Color("blue"),len(depths)))
        hex_colors = [j.hex for j in colors]
        
        for idx_profs, z in enumerate(depths):
            # cut it off at depth (dBars) >= z
            df_tmp = profiles_subset[profiles_subset['depth (dBars)'] >= z] 
            df_tmp_grouped = df_tmp.groupby('profile_num').median()
            df_tmp_grouped['datetime'] = pd.to_datetime('2021-1-1') + pd.to_timedelta(df_tmp_grouped.time, unit='D') - pd.Timedelta(days=1)
            df_tmp_grouped = df_tmp_grouped.sort_values('dist')


            ax.plot(df_tmp_grouped.dist,df_tmp_grouped['chla (ppb)'], label='chla mean(0'+str(z)+')', color=hex_colors[idx_profs], ls='--')
            ax.scatter(df_tmp_grouped.dist,df_tmp_grouped['chla (ppb)'], color=hex_colors[idx_profs], ls='--')
        
        ax.set_ylim(-0.3,0.6)
        
        if idx not in [5,28,39,40,41]:
            ax.set_xlim(dist.max(),0)
        else:
            ax.set_xlim(0,dist.max())


        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
       

        ########################################

        fig, ax = plt.subplots(1,4, figsize=(30,5))


        vMag_reshape = vMag.reshape(-1,69).T
        vDir_reshape = vDir.reshape(-1,69).T
        vShear_reshape = vShear.reshape(-1,68).T


        x, y = np.meshgrid(dist,Y[:69,0])    
        im = ax[0].pcolormesh(x,y,vMag_reshape,shading='gouraud', vmin=0,vmax=200)
        fig.colorbar(im,ax=ax[0])
        ax[0].set_title("Current Magnitude (cm/s)")
        ax[0].set_ylabel('Depth (m)')
        ax[0].set_xlabel('Dist from Start (m)')

        im = ax[1].pcolormesh(x,y,vDir_reshape,shading='gouraud', vmin=0,vmax=360, cmap='hsv')
        fig.colorbar(im,ax=ax[1])
        ax[1].set_title("Current Direction (Deg from North)")
        ax[1].set_ylabel('Depth (m)')
        ax[1].set_xlabel('Dist from Start (m)')
                
        vgradient = []
        
        N=180
        current_speed = np.mean(vMag_reshape[0:4,:],axis=0)
        current_speed_smooth = pd.Series(current_speed).rolling(window=N).mean().iloc[N-1:].values
        vgradient = []

        step = 1

        for i in range(len(current_speed)-N):
            du = current_speed_smooth[i] - current_speed_smooth[i+step]
            dx = dist[i] - dist[i+step]
            current_grad = du/dx
            vgradient.append(current_grad)
            
        dudx = scipy.signal.savgol_filter(current_speed_smooth, window_length=11, polyorder=2, deriv=1)

#         ax_twin = ax[0].twinx()
#         ax_twin.plot(dist[:-N+1], dudx, c='grey')
#         ax_twin.set_ylim(-0.6,0.6)
        
#         ax_twin2 = ax[0].twinx()
#         ax_twin2.plot(dist[:-N],current_speed_smooth[:-1],c='black')
#         ax_twin2.set_ylim(0,200)
        
        Ndir=90
        current_dir = np.mean(vDir_reshape[0:4,:],axis=0)
        current_dir_smooth = pd.Series(current_dir).rolling(window=Ndir).mean().iloc[Ndir-1:].values
        
#         ax_dir = ax[1].twinx()
#         ax_dir.plot(dist[:-Ndir+1],current_dir_smooth)
#         ax_dir.set_ylim(0,360)

        # taking off the first 100 points because those are often noisy and never the front
        front_location = np.argmax(np.abs(dudx[100:-30]))+100

        echo_reshape = echo.reshape(-1,127).T
        
        x_trim = dist.shape[0] - echo_reshape.shape[1]

        x, y = np.meshgrid(dist[:,0],Yecho[:127,0])      
        
        # this is actually maybe stupid so need to pull in the shear X and Y data
        if x_trim >=0:
            im = ax[3].pcolormesh(x[:,x_trim:],y[:,x_trim:],echo_reshape,shading='gouraud', vmin=96,vmax=115, cmap='jet')
        else:
            im = ax[3].pcolormesh(x,y,echo_reshape[:,:x_trim],shading='gouraud', vmin=96,vmax=115, cmap='jet')
        
        
#         im = ax[3].pcolormesh(x,y,echo_reshape,shading='gouraud', vmin=96,vmax=115, cmap='jet')
        fig.colorbar(im,ax=ax[3])
#         ax[3].axvline(dist[front_location],color='grey', ls='--')
        ax[3].set_title("Echo Volume Backscatter (sV)")
        ax[3].set_ylabel('Depth (m)')
        ax[3].set_xlabel('Dist from Start (m)')
        ax[3].set_ylim(-100,-1)
        
        if idx not in [5,28,39,40,41]:
            for i in range(4):
                ax[i].set_xlim(dist.max(),0)    
        else:
            for i in range(4):
                ax[i].set_xlim(0,dist.max())
        for i in range(4):
            ax[i].set_ylim(-100,-1)
#             ax[i].axvline(dist[front_location],color='grey', ls='--', label='current front')
            
#             ax[i].scatter(dist[bad_idxs],[-68]*len(dist[bad_idxs]),color='red')
# #             for sal_front in sal_front_location:
#                 ax[i].axvline(sal_front, c='k', ls='--', label='sal front')
#         ax[0].legend(loc='lower right')

    #     fig.savefig('sst_mag_dir_'+str(transect_num[idx])+'.png',dpi=400)
#         fig.tight_layout()

        x_trim = dist.shape[0] - vShear_reshape.shape[1]

        x, y = np.meshgrid(dist[:,0],Y[:68,0])  
        
        # this is actually maybe stupid so need to pull in the shear X and Y data
        if x_trim >=0:
            im = ax[2].pcolormesh(x[:,x_trim:],y,vShear_reshape,shading='gouraud', vmin=0.01,vmax=.09, cmap='cividis')
        else:
            im = ax[2].pcolormesh(x,y,vShear_reshape[:,:x_trim],shading='gouraud', vmin=0.01,vmax=.09, cmap='cividis')
        fig.colorbar(im,ax=ax[2])
        ax[2].set_title("Current Shear")
        ax[2].set_ylabel('Depth (m)')
        ax[2].set_xlabel('Dist from Start (m)')
        
        plt.savefig('adcp'+str(idx)+'.png',dpi=300)
        
        plt.show()
        
        ###########
        
        profiles_subset = profiles.loc[timestamps[0]:timestamps[-1]]
    
        profiles_subset = profile_locs(profiles_subset, adcp_df)
        
                
        # plot T and S diagrams first with distance from coast and then with other variables
        fig, ax = plt.subplots(1,4, figsize=(30,5))
        markersize=40

        for i in range(4):
            ax[i].set_xlim(32,37)
            ax[i].set_ylim(8,30)
            ax[i].set_xlabel('Salinity')
            ax[i].set_ylabel('Temperature (C)')

            CS = ax[i].contour(si,ti,dens, linestyles='dashed', colors='k', alpha=0.5)
            ax[i].clabel(CS, fontsize=12, inline=1, fmt='%1.1f') # Label every second level
            # ellipse = Ellipse(xy=(35.88, 28.05), width=0.5, height=1.2, angle=-20,
            #                 edgecolor='k', fc='None', ls='--', lw=2)
            # ax.add_patch(ellipse)

        # im = ax.scatter(np.concatenate(vmp_sal_list), np.concatenate(vmp_temp_list), c=np.concatenate(vmp_chla_list), vmin=-0.5,vmax=0.5,alpha=0.4)#c=np.concatenate(vmp_dt_list), vmin=0,vmax=365, cmap='twilight')
        im = ax[0].scatter(profiles_subset['salinity (PSU)'], profiles_subset['temp (C)'], c=profiles_subset['depth (dBars)'], cmap=cmocean.cm.deep_r, alpha=1, vmin=-70,vmax=0)
        fig.colorbar(im,ax=ax[0])
        ax[0].set_title('Profile Depth')
        im = ax[1].scatter(profiles_subset['salinity (PSU)'], profiles_subset['temp (C)'], c=profiles_subset['chla (ppb)'], cmap=cmocean.cm.algae,
                          vmax=np.percentile(profiles_subset['chla (ppb)'], 95),vmin=np.percentile(profiles_subset['chla (ppb)'], 5), alpha=1)
        fig.colorbar(im,ax=ax[1])
        ax[1].set_title('Profile Chla')
        im = ax[2].scatter(profiles_subset['salinity (PSU)'], profiles_subset['temp (C)'], c=profiles_subset['turbidity (FTU)'], cmap=cmocean.cm.turbid, 
                           vmax=np.percentile(profiles_subset['turbidity (FTU)'], 95),vmin=np.percentile(profiles_subset['turbidity (FTU)'], 5), alpha=1)
        fig.colorbar(im,ax=ax[2])
        ax[2].set_title('Profile Turbidity')
        
#         im = ax[3].scatter(profiles['salinity (PSU)'], profiles['temp (C)'], c=profiles.datetime.dt.day_of_year, cmap=cmocean.cm.phase,vmin=0,vmax=365,alpha=0.8)
#         im = ax[3].scatter(profs_turb_sub['salinity (PSU)'], profs_turb_sub['temp (C)'], c=profs_turb_sub['cluster'], cmap='jet',vmin=0,vmax=6,alpha=0.05)
#         fig.colorbar(im,ax=ax[3])
# #         ax[3].set_title('All Transects/Profiles DOY')
#         ax[3].set_title('All Profiles Clusters')
        
        plt.show()
        
        ######
        #### calc decorrelation scales

#         fig, ax = plt.subplots(1,3,figsize=(30,10))

#         depth_max = [0,-15,-25,-35]
#         depth_min = [-10,-25,-35,-45]
#         vars_for_calc = ['salinity (PSU)','current_mag','ekbackscatter']

#         for var_i, ax in enumerate([ax[0],ax[1],ax[2]]):

#             var_for_cal = vars_for_calc[var_i]

#             ax2=ax.twinx()
#             ax.axhline(0,ls='--', c='k')
#             for depth_i in range(len(depth_max)):
#                 input_df = profiles_subset[(profiles_subset['depth (dBars)'] < depth_max[depth_i]) & (profiles_subset['depth (dBars)'] > depth_min[depth_i])].groupby('profile_num',dropna=False).mean()
#                 input_df = input_df.reindex(index=natsorted(input_df.index))
#                 nlags = len(input_df)-1

#                 ax2.plot(input_df['dist'],input_df[var_for_cal],label=var_for_cal+' '+str(depth_max[depth_i]) + ' to ' +str(depth_min[depth_i]),ls='--')
#                 ax.plot(input_df['dist'],sm.tsa.acf(input_df[var_for_cal],nlags=nlags), label='autocorr '+var_for_cal+' '+str(depth_max[depth_i]) + ' to ' +str(depth_min[depth_i]))
#                 ax.set_title(var_for_cal)
#                 ax.legend(loc='lower left',bbox_to_anchor=(-0.1, -0.25))
#                 ax2.legend(loc='lower right',bbox_to_anchor=(1.05, -0.25))
#         plt.show()
        
#         fig, ax = plt.subplots()
#         p_sub = profiles_subset[profiles_subset['depth (dBars)'] > -45]
#         correls = []
#         for p in p_sub.profile_num.unique():
#             correls.append([crosscorr(p_sub[p_sub.profile_num == p].chla_clean, p_sub[p_sub.profile_num == p].ekbackscatter, lag=i) for i in range(50)])
#         correls = np.array(correls)
#         ax.scatter([list(range(50))]*correls.shape[0],correls,alpha=0.4)
#         plt.show()


        #########################
#         calculate the watermass backscatter values
#         if idx == 39:
#             gulf_stream_ek.append(echo_reshape[:ed_gs_depth,:].flatten())
#         elif idx == 40:
#             shelf_dist_arg = (np.abs(dist - 4000)).argmin()
#             shelf_ek.append(echo_reshape[:shelf_depth,:shelf_dist_arg].flatten())                   
#             eddy_ek.append(echo_reshape[:ed_gs_depth,shelf_dist_arg:].flatten())
#         elif idx == 41:
#             shelf_dist_arg = (np.abs(dist - 4000)).argmin()
#             shelf_ek.append(echo_reshape[:shelf_depth,:shelf_dist_arg].flatten())
#             gs_start_dist_arg = (np.abs(dist - 9000)).argmin()
#             gs_end_dist_arg = (np.abs(dist - 13000)).argmin()
#             gulf_stream_ek.append(echo_reshape[:ed_gs_depth,gs_start_dist_arg:gs_end_dist_arg].flatten())
#         elif idx == 42:
#             ed_start_dist_arg = (np.abs(dist - 3000)).argmin()
#             ed_end_dist_arg = (np.abs(dist - 21000)).argmin()
#             eddy_ek.append(echo_reshape[:ed_gs_depth,ed_start_dist_arg:ed_end_dist_arg].flatten())

        ########################################
        
#         fig, ax = plt.subplots(1,3, figsize=(30,5))
#         im = ax[0].scatter(long,lat,c=sst[:,0], cmap='inferno')
#         fig.colorbar(im,ax=ax[0])
#         ax[0].scatter(long[front_location],lat[front_location],s=100,c='grey')
#         ax[0].set_title("Location (and SST)")

#         echo_reshape = echo.reshape(-1,127).T
        
#         Ndir=90
#         current_dir = np.mean(vDir_reshape[0:4,:],axis=0)
#         current_dir_smooth = pd.Series(current_dir).rolling(window=Ndir).mean().iloc[Ndir-1:].values

#         x, y = np.meshgrid(Xecho.reshape(-1,127)[:,0],Yecho[:127,0])    
#         im = ax[1].pcolormesh(x,y,echo_reshape,shading='gouraud', vmin=96,vmax=115, cmap='jet')
#         fig.colorbar(im,ax=ax[1])
#         ax[1].axvline(dist[front_location],color='grey', ls='--')
#         ax[1].set_title("Echo Volume Backscatter (sV)")
#         ax[1].set_ylabel('Depth (m)')
#         ax[1].set_xlabel('Dist from Start (m)')
#         ax[1].set_ylim(-70,-1)
        
       
#         ax[2].plot(dist,sst, color='red')
# #         ax[0].set_ylim(15,30)
#         ax[2].set_title("SST (C) and Dir (from 10 m)")
#         ax[2].set_ylabel('Temp (C)')
#         ax[2].set_xlabel('Dist from Start (m)')
#         ax[2].set_ylim(15,30)
#         ax[2].axvline(dist[front_location],color='grey', ls='--')
        
#         ax_dir = ax[2].twinx()
#         ax_dir.plot(dist[:-Ndir+1],current_dir_smooth)
#         ax_dir.set_ylim(0,360)
        
#         if idx in [5,28,39,40,41]:
#             for i in range(1,3):
#                 ax[i].set_xlim(dist.max(),0)
# #         fig.tight_layout()
#         plt.show()
        
        


    except Exception as e: 
        print('had an error')
        print(e)
        
    print('##################################################################################################################################################')
    print('##################################################################################################################################################')
    print('##################################################################################################################################################')
    print('##################################################################################################################################################')
#         print('##################################################################################################################################################')
#         print('##################################################################################################################################################')

In [ ]:
#The transects that seem to actually be in the GS and cross a current front are:
    
gs_current_idxs = [68, 67, 66, 65, 64, 63, 59, 58, 56, 52, 51, 50, 49, 48, 46, 45, 44, 43, 38, 36, 35, 34, 30, 29, 25, 22, 20, 17, 16, 13, 12, 10, 9, 8, 7, 6]

# steps

1. grab the data from these transects
2. detect the main current front
3. line them up in space with the current front as 0

In [ ]:
np.argmax(np.abs(dudx))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5.5,5))

profiles_subset = profiles.loc[timestamps[0]:timestamps[-1]]

profiles_subset = profile_locs(profiles_subset, adcp_df)

#         integrate down to z depth
depths = [-5,-10,-20,-40,-60]

colors = list(Color("black").range_to(Color("green"),len(depths)))
hex_colors = [j.hex for j in colors]

colors = list(Color("red").range_to(Color("yellow"),len(depths)))
hex_colors_red = [j.hex for j in colors]

#     dist_centered = dist - single_front_locations[counter]

ax_mag = ax.twinx()

for idx_profs, z in enumerate(depths):
    # cut it off at depth (dBars) >= z
    df_tmp = profiles_subset[profiles_subset['depth (dBars)'] >= z] 
    df_tmp_grouped = df_tmp.groupby('profile_num').median()
    df_tmp_grouped['datetime'] = pd.to_datetime('2021-1-1') + pd.to_timedelta(df_tmp_grouped.time, unit='D') - pd.Timedelta(days=1)
    df_tmp_grouped = df_tmp_grouped.sort_values('dist')


    ax.plot(df_tmp_grouped.dist,df_tmp_grouped['chla (ppb)'], label='chla mean(0'+str(z)+')', color=hex_colors[idx_profs], ls='-')
    ax.scatter(df_tmp_grouped.dist,df_tmp_grouped['chla (ppb)'], color=hex_colors[idx_profs], ls='--')
    
    ax_mag.plot(df_tmp_grouped.dist,df_tmp_grouped['current_mag'], label='mag mean(0'+str(z)+')', color=hex_colors_red[idx_profs], ls='--')
    ax_mag.scatter(df_tmp_grouped.dist,df_tmp_grouped['current_mag'], color=hex_colors_red[idx_profs], ls='--')
    
    if idx_profs == 0:

        dudx = scipy.signal.savgol_filter(df_tmp_grouped['current_mag'], window_length=3, polyorder=2, deriv=1)
        front_location = np.argmax(np.abs(dudx))
        ax.axvline((df_tmp_grouped.dist[front_location]+df_tmp_grouped.dist[front_location+1])/2,color='k', ls='--')
            
    ax.set_ylim(-0.3,0.8)
    ax.set_ylabel('chla')
       

In [ ]:
profiles[profiles['chla (ppb)']>5]['chla (ppb)']

In [ ]:
plt.hist(profiles[profiles['chla (ppb)']<2]['chla (ppb)'],bins=50)
plt.show()

In [ ]:
profiles['chla (ppb)'].min()

In [ ]:
profiles['chla_adjusted'] = profiles['chla (ppb)']+.78

In [ ]:
profiles['chla_adjusted'].min()

In [ ]:
plt.hist(profiles[profiles['chla_adjusted']<2]['chla_adjusted'],bins=50)
plt.show()

In [ ]:
profiles[profiles['chla (ppb)']<2]['chla (ppb)']

In [ ]:
dist.shape

In [ ]:
current_speed_smooth.shape

In [ ]:
y_fit.shape

In [ ]:
ig, ax = plt.subplots(5,2, figsize=(16, 14))
i = 0
 
# define window sizes 5, 11, 21, 31
for poly in [1,2,3,4,5]:    
    y_fit = scipy.signal.savgol_filter(current_speed_smooth, 501, poly, mode="nearest")
    ax[i,0].plot(dist[:-N+1],current_speed_smooth, label="y_signal", color="green")
    ax[i,1].plot(dist[:-N+1], y_fit, label="y_smoothed", color="red")
    for j in range(2):
        ax[i,j].set_title("poly order: " + str(poly))
        ax[i,j].legend()
        ax[i,j].grid(True)
    i+=1

plt.tight_layout()        
plt.show() 

In [ ]:
for idx,fn in enumerate(filenames):
    if idx not in gs_current_idxs:
        continue
#     if idx != 64:
#         continue
    print(idx)
    print(fn)
    
#     try:

    dist = scipy.io.loadmat(fn+'_distance.mat')['distance']
    sst = scipy.io.loadmat(fn+'_SST.mat')['SST']
    time = scipy.io.loadmat(fn+'_time.mat')['time']
    timestamps = pd.to_datetime(time[0,:]-719529, unit='D')

    X = scipy.io.loadmat(fn+'_X.mat')['X']
    Y = scipy.io.loadmat(fn+'_Y.mat')['Y']
    vMag = scipy.io.loadmat(fn+'_vmag.mat')['C']

    vDir = scipy.io.loadmat(fn+'_vdir.mat')['C']

    echo = scipy.io.loadmat(fn+'_echo.mat')['C']
    Xecho = scipy.io.loadmat(fn+'_Xecho.mat')['X']
    Yecho = scipy.io.loadmat(fn+'_Yecho.mat')['Y']
    # correcting for the smaller bin size
    Yecho = (Yecho*0.502)-1

    lat = scipy.io.loadmat(fn+'_lat.mat')['lat']
    long = scipy.io.loadmat(fn+'_long.mat')['long']

    # the distances are crazy wrong sometimes so re-calcuating the distance
    coords_1 = (lat[0], long[0])
    distances = []
    for i in range(len(lat)):
        coords_2 = (lat[i], long[i])
        distances.append(distance.geodesic(coords_1, coords_2).meters)
    dist = np.array(distances)       
#         bogus_dist_locations = dist > 35000        
#         dist[dist > 25000] = np.nan
#         dist = pd.Series(dist).interpolate().values
    # checking if the current point is within 500 of one that was good, assuming the first point is good
    # then if not setting it to nan and interpolating it out
    last_good_idx = 0
    bad_idxs = []
    for di,d in enumerate(dist):
        if dist[di] - dist[last_good_idx] < 500:
            last_good_idx = di
        else:
            bad_idxs.append(di)
    dist[bad_idxs] = np.nan
    dist = pd.Series(dist).interpolate().values

    dist = dist.reshape(-1,1)
    
    adcp_df = pd.DataFrame({'dt':timestamps,
                    'datetime':timestamps,
                   'lat': lat[:,0],
                   'lon':long[:,0],
                           'dist':dist[:,0]})

    adcp_df = adcp_df.set_index('dt')
    adcp_df = adcp_df.sort_index(ascending=True)

    ax = plot_profiles(profiles, adcp_df, timestamps[0],timestamps[-1], ['potential density (kg/m^3 -1000)','temp (C)', 'salinity (PSU)', 'chla (ppb)'], plot=True)
    if idx not in [5,28,39,40,41]:
        for i in range(4):
            ax[i].set_xlim(dist.max(),0)    
    else:
        for i in range(4):
            ax[i].set_xlim(0,dist.max())
    for i in range(4):
        ax[i].set_ylim(-80,-1)
#             for sal_front in sal_front_location:
#                 ax[i].axvline(sal_front, c='k', ls='--')
#         plt.savefig('profiles'+str(idx)+'.png',dpi=300)
    plt.show()

    fig, ax = plt.subplots(1,1, figsize=(9.5,4))
    vMag_reshape = vMag.reshape(-1,69).T
    vDir_reshape = vDir.reshape(-1,69).T
    
    x, y = np.meshgrid(dist,Y[:69,0])  
    im = ax.pcolormesh(x,y,vMag_reshape,shading='gouraud', vmin=0,vmax=250)
    fig.colorbar(im,ax=ax)
    ax.set_title("Current Magnitude (cm/s)")

    vgradient = []

    N=180
    current_speed = np.mean(vMag_reshape[0:4,:],axis=0)
    current_speed_smooth = pd.Series(current_speed).rolling(window=N).mean().iloc[N-1:].values
    vgradient = []

    step = 1

    for i in range(len(current_speed)-N):
        du = current_speed_smooth[i] - current_speed_smooth[i+step]
        dx = dist[i] - dist[i+step]
        current_grad = du/dx
        vgradient.append(current_grad)

    dudx = scipy.signal.savgol_filter(current_speed_smooth, window_length=501, polyorder=2, deriv=1)

    ax_twin = ax.twinx()
#         ax_twin.plot(dist[15:-15-N], vgradient_smooth[15:-16], c='k', alpha=0.5)
    ax_twin.plot(dist[:-N+1], dudx, c='grey', ls='--')
    ax_twin.set_ylim(-0.6,0.6)

    ax_twin2 = ax.twinx()
    ax_twin2.plot(dist[:-N],current_speed_smooth[:-1],c='black')
    ax_twin2.set_ylim(0,250)

    ax_twin3 = ax.twinx()
    ax_twin3.plot(dist,sst, color='red')
    ax_twin3.set_ylim(18,30)
    # taking off the first 100 points because those are often noisy and never the front
    front_location = np.argmax(np.abs(dudx[100:-30]))+100

    ax_twin.yaxis.label.set_color('grey')
    ax_twin.spines['right'].set_position(('outward', 90))

    ax_twin2.yaxis.label.set_color('black')
    ax_twin2.spines['right'].set_position(('outward', 180))

    ax_twin3.yaxis.label.set_color('red')
    ax_twin3.spines['right'].set_position(('outward', 260))

    ax_twin.set_ylabel('current gradient')
    ax_twin2.set_ylabel('top 10m current speed (cm/s)')
    ax_twin3.set_ylabel('SST (C)')


    if idx not in [5,28,39,40,41]:
        ax.set_xlim(dist.max(),0)
    else:
        ax.set_xlim(0,dist.max())
    ax.axvline(dist[front_location],color='k', ls='--')
    ax.set_ylim(-80,-2)
    ax.set_ylabel('Depth (m)')
    ax.set_xlabel('Dist from Start (m)')
    ax.axvline(dist[front_location],color='k', ls='--')


    fig.tight_layout()
#         fig.savefig('mag_dir_echo'+str(idx)+'.png',dpi=400)
    plt.show()
    
    ##################################
    
    fig, ax = plt.subplots(1,1, figsize=(5.5,5))
    
    ax.axvline(dist[front_location],color='k', ls='--')


    profiles_subset = profiles.loc[timestamps[0]:timestamps[-1]]

    profiles_subset = profile_locs(profiles_subset, adcp_df)

    #         integrate down to z depth
    depths = [-10,-20,-40,-60]

    colors = list(Color("black").range_to(Color("green"),len(depths)))
    hex_colors = [j.hex for j in colors]

    colors = list(Color("red").range_to(Color("yellow"),len(depths)))
    hex_colors_red = [j.hex for j in colors]

    #     dist_centered = dist - single_front_locations[counter]

    ax_mag = ax.twinx()

    for idx_profs, z in enumerate(depths):
        # cut it off at depth (dBars) >= z
        df_tmp = profiles_subset[profiles_subset['depth (dBars)'] >= z] 
        df_tmp_grouped = df_tmp.groupby('profile_num').median()
        df_tmp_grouped['datetime'] = pd.to_datetime('2021-1-1') + pd.to_timedelta(df_tmp_grouped.time, unit='D') - pd.Timedelta(days=1)
        df_tmp_grouped = df_tmp_grouped.sort_values('dist')


        ax.plot(df_tmp_grouped.dist,df_tmp_grouped['chla_adjusted']/df_tmp_grouped['chla_adjusted'].mean(), label='chla mean(0'+str(z)+')', color=hex_colors[idx_profs], ls='-')
        ax.scatter(df_tmp_grouped.dist,df_tmp_grouped['chla_adjusted']/df_tmp_grouped['chla_adjusted'].mean(), color=hex_colors[idx_profs], ls='--')

#         ax_mag.plot(df_tmp_grouped.dist,df_tmp_grouped['current_mag'], label='mag mean(0'+str(z)+')', color=hex_colors_red[idx_profs], ls='--')
#         ax_mag.scatter(df_tmp_grouped.dist,df_tmp_grouped['current_mag'], color=hex_colors_red[idx_profs], ls='--')

#         if idx_profs == 1:
#             dudx = scipy.signal.savgol_filter(df_tmp_grouped['current_mag'], window_length=3, polyorder=2, deriv=1)
#             front_location = np.argmax(np.abs(dudx))
#             print(front_location)
#             ax.axvline(df_tmp_grouped.dist[front_location],color='grey', ls='--')

#     ax.set_ylim(-0.3,0.8)
    ax.set_ylabel('chla')
    
    if idx not in [5,28,39,40,41]:
        ax.set_xlim(dist.max(),0)
    else:
        ax.set_xlim(0,dist.max())

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    plt.show()
    
    #############################################

    print('--------------------------------------')

#     except Exception as e: 
#         print(e)
    

Now we just want to plot all of them together on the same figure centered at the front

In [1]:
gs_current_idxs = [68, 67, 66, 65, 64, 63, 59, 58, 56, 52, 51, 49, 48, 46, 45, 44, 43, 38, 34, 30, 22, 20, 17, 16, 13, 12, 10, 9, 8, 7, 6]

In [2]:
len(gs_current_idxs)

31

In [ ]:
df_tmp_grouped

In [ ]:
df_tmp_grouped.dist

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(16,50))

for idx,fn in enumerate(filenames):
    if idx not in gs_current_idxs:
        continue
#     if idx != 64:
#         continue
    print(idx)
    print(fn)
    
#     try:

    dist = scipy.io.loadmat(fn+'_distance.mat')['distance']
    sst = scipy.io.loadmat(fn+'_SST.mat')['SST']
    time = scipy.io.loadmat(fn+'_time.mat')['time']
    timestamps = pd.to_datetime(time[0,:]-719529, unit='D')

    X = scipy.io.loadmat(fn+'_X.mat')['X']
    Y = scipy.io.loadmat(fn+'_Y.mat')['Y']
    vMag = scipy.io.loadmat(fn+'_vmag.mat')['C']

    vDir = scipy.io.loadmat(fn+'_vdir.mat')['C']

    echo = scipy.io.loadmat(fn+'_echo.mat')['C']
    Xecho = scipy.io.loadmat(fn+'_Xecho.mat')['X']
    Yecho = scipy.io.loadmat(fn+'_Yecho.mat')['Y']
    # correcting for the smaller bin size
    Yecho = (Yecho*0.502)-1

    lat = scipy.io.loadmat(fn+'_lat.mat')['lat']
    long = scipy.io.loadmat(fn+'_long.mat')['long']

    # the distances are crazy wrong sometimes so re-calcuating the distance
    coords_1 = (lat[0], long[0])
    distances = []
    for i in range(len(lat)):
        coords_2 = (lat[i], long[i])
        distances.append(distance.geodesic(coords_1, coords_2).meters)
    dist = np.array(distances)       
#         bogus_dist_locations = dist > 35000        
#         dist[dist > 25000] = np.nan
#         dist = pd.Series(dist).interpolate().values
    # checking if the current point is within 500 of one that was good, assuming the first point is good
    # then if not setting it to nan and interpolating it out
    last_good_idx = 0
    bad_idxs = []
    for di,d in enumerate(dist):
        if dist[di] - dist[last_good_idx] < 500:
            last_good_idx = di
        else:
            bad_idxs.append(di)
    dist[bad_idxs] = np.nan
    dist = pd.Series(dist).interpolate().values

    dist = dist.reshape(-1,1)
    
    adcp_df = pd.DataFrame({'dt':timestamps,
                    'datetime':timestamps,
                   'lat': lat[:,0],
                   'lon':long[:,0],
                           'dist':dist[:,0]})

    adcp_df = adcp_df.set_index('dt')
    adcp_df = adcp_df.sort_index(ascending=True)

#     ax = plot_profiles(profiles, adcp_df, timestamps[0],timestamps[-1], ['potential density (kg/m^3 -1000)','temp (C)', 'salinity (PSU)', 'chla (ppb)'], plot=True)
#     if idx not in [5,28,39,40,41]:
#         for i in range(4):
#             ax[i].set_xlim(dist.max(),0)    
#     else:
#         for i in range(4):
#             ax[i].set_xlim(0,dist.max())
#     for i in range(4):
#         ax[i].set_ylim(-80,-1)
# #             for sal_front in sal_front_location:
# #                 ax[i].axvline(sal_front, c='k', ls='--')
# #         plt.savefig('profiles'+str(idx)+'.png',dpi=300)
#     plt.show()

#     fig, ax = plt.subplots(1,1, figsize=(9.5,4))
    vMag_reshape = vMag.reshape(-1,69).T
    vDir_reshape = vDir.reshape(-1,69).T
    
#     x, y = np.meshgrid(dist,Y[:69,0])  
#     im = ax.pcolormesh(x,y,vMag_reshape,shading='gouraud', vmin=0,vmax=250)
#     fig.colorbar(im,ax=ax)
#     ax.set_title("Current Magnitude (cm/s)")

    vgradient = []

    N=180
    current_speed = np.mean(vMag_reshape[0:4,:],axis=0)
    current_speed_smooth = pd.Series(current_speed).rolling(window=N).mean().iloc[N-1:].values

    step = 1

    for i in range(len(current_speed)-N):
        du = current_speed_smooth[i] - current_speed_smooth[i+step]
        dx = dist[i] - dist[i+step]

    dudx = scipy.signal.savgol_filter(current_speed_smooth, window_length=501, polyorder=2, deriv=1)

#     ax_twin = ax.twinx()
#     ax_twin.plot(dist[:-N+1], dudx, c='grey', ls='--')
#     ax_twin.set_ylim(-0.6,0.6)

#     ax_twin2 = ax.twinx()
#     ax_twin2.plot(dist[:-N],current_speed_smooth[:-1],c='black')
#     ax_twin2.set_ylim(0,250)

#     ax_twin3 = ax.twinx()
#     ax_twin3.plot(dist,sst, color='red')
#     ax_twin3.set_ylim(18,30)
#     # taking off the first 100 points because those are often noisy and never the front
    front_location = np.argmax(np.abs(dudx[100:-30]))+100

#     ax_twin.yaxis.label.set_color('grey')
#     ax_twin.spines['right'].set_position(('outward', 90))

#     ax_twin2.yaxis.label.set_color('black')
#     ax_twin2.spines['right'].set_position(('outward', 180))

#     ax_twin3.yaxis.label.set_color('red')
#     ax_twin3.spines['right'].set_position(('outward', 260))

#     ax_twin.set_ylabel('current gradient')
#     ax_twin2.set_ylabel('top 10m current speed (cm/s)')
#     ax_twin3.set_ylabel('SST (C)')


#     if idx not in [5,28,39,40,41]:
#         ax.set_xlim(dist.max(),0)
#     else:
#         ax.set_xlim(0,dist.max())
#     ax.axvline(dist[front_location],color='k', ls='--')
#     ax.set_ylim(-80,-2)
#     ax.set_ylabel('Depth (m)')
#     ax.set_xlabel('Dist from Start (m)')
#     ax.axvline(dist[front_location],color='k', ls='--')


#     fig.tight_layout()
# #         fig.savefig('mag_dir_echo'+str(idx)+'.png',dpi=400)
#     plt.show()
    
    ##################################
    
    
    
#     ax.axvline(dist[front_location],color='k', ls='--')


    profiles_subset = profiles.loc[timestamps[0]:timestamps[-1]]

    profiles_subset = profile_locs(profiles_subset, adcp_df)

    #         integrate down to z depth
    depths = [-10,-20,-30,-40,-60]

    colors = list(Color("black").range_to(Color("blue"),len(depths)))
    hex_colors = [j.hex for j in colors]

    for idx_profs, z in enumerate(depths):
        # cut it off at depth (dBars) >= z
        df_tmp = profiles_subset[profiles_subset['depth (dBars)'] >= z] 
        df_tmp_grouped = df_tmp.groupby('profile_num').median()
        df_tmp_grouped['datetime'] = pd.to_datetime('2021-1-1') + pd.to_timedelta(df_tmp_grouped.time, unit='D') - pd.Timedelta(days=1)
        df_tmp_grouped = df_tmp_grouped.sort_values('dist')
        
        # here we're using the closest profile value to the front to normalize the transect 
        front_chla = df_tmp_grouped.iloc[(df_tmp_grouped['dist']-dist[front_location]).abs().argsort()[:1]]['chla_adjusted'].values

        ax[idx_profs].plot(df_tmp_grouped.dist-dist[front_location],df_tmp_grouped['chla_adjusted']/df_tmp_grouped['chla_adjusted'][-1], label='chla mean(0'+str(z)+')', color=hex_colors[idx_profs], ls='-')
        ax[idx_profs].scatter(df_tmp_grouped.dist-dist[front_location],df_tmp_grouped['chla_adjusted']/df_tmp_grouped['chla_adjusted'][-1], color=hex_colors[idx_profs], ls='--')
        
        ax[idx_profs].set_title(z)

#         ax_mag.plot(df_tmp_grouped.dist,df_tmp_grouped['current_mag'], label='mag mean(0'+str(z)+')', color=hex_colors_red[idx_profs], ls='--')
#         ax_mag.scatter(df_tmp_grouped.dist,df_tmp_grouped['current_mag'], color=hex_colors_red[idx_profs], ls='--')

#         if idx_profs == 1:
#             dudx = scipy.signal.savgol_filter(df_tmp_grouped['current_mag'], window_length=3, polyorder=2, deriv=1)
#             front_location = np.argmax(np.abs(dudx))
#             print(front_location)
#             ax.axvline(df_tmp_grouped.dist[front_location],color='grey', ls='--')

#     ax.set_ylim(-0.3,0.8)
for i in range(5):
    ax[i].axvline(0,color='grey', ls='--')
    ax[i].set_ylabel('chla normalized to mean')
    ax[i].set_xlim(7000,-7000)
    ax[i].set_ylim(0.25,1.75)
    # ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
plt.savefig('chla_across_front_normed_to_coastal_point.png',dpi=300)
plt.show()
    
    #############################################


#     except Exception as e: 
#         print(e)
    

In [ ]:
plt.hist([d.timetuple().tm_yday for d in datetimes_subset],bins=15)

In [ ]:
plt.hist([d.timetuple().tm_yday for d in datetimes_subset],bins=15)

In [ ]:
! pip install statsmodels

In [ ]:
profiles_sub

In [ ]:
min(10 * np.log10(16), 16 - 1)

In [ ]:
from natsort import natsorted

In [ ]:
profiles_subset[profiles_subset['depth (dBars)'] > -8].groupby('profile_num').mean()

In [ ]:
len(profiles_subset[profiles_subset['depth (dBars)'] > -8].groupby('profile_num').mean()['dist'])

In [ ]:
len(profiles_subset[profiles_subset['depth (dBars)'] > -8].groupby('profile_num').mean()['salinity (PSU)'])

In [ ]:
len(sm.tsa.acf(profiles_subset[profiles_subset['depth (dBars)'] > -8].groupby('profile_num').mean()['salinity (PSU)']))

In [ ]:
vmag_for_profile

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(30,10))

depth_max = [0,-15,-25,-35]
depth_min = [-10,-25,-35,-45]
vars_for_calc = ['salinity (PSU)','temp (C)','chla (ppb)']

for var_i, ax in enumerate([ax[0],ax[1],ax[2]]):
    
    var_for_cal = vars_for_calc[var_i]

    ax2=ax.twinx()
    ax.axhline(0,ls='--', c='k')
    for depth_i in range(len(depth_max)):
        input_df = profiles_subset[(profiles_subset['depth (dBars)'] < depth_max[depth_i]) & (profiles_subset['depth (dBars)'] > depth_min[depth_i])].groupby('profile_num').mean()
        input_df = input_df.reindex(index=natsorted(input_df.index))
        nlags = len(input_df)-1

        ax2.plot(input_df['dist'],input_df[var_for_cal],label=var_for_cal+' '+str(depth_max[depth_i]) + ' to ' +str(depth_min[depth_i]),ls='--')
        ax.plot(input_df['dist'],sm.tsa.acf(input_df[var_for_cal],nlags=nlags), label='autocorr '+var_for_cal+' '+str(depth_max[depth_i]) + ' to ' +str(depth_min[depth_i]))
        ax.set_title(var_for_cal)
        ax.legend(loc='lower left',bbox_to_anchor=(-0.1, -0.25))
        ax2.legend(loc='lower right',bbox_to_anchor=(1.05, -0.25))

In [ ]:
vMag_reshape[:10,:].mean(axis=0).shape

In [ ]:
x[0,:]

In [ ]:
import statsmodels.api as sm


# input_df = profiles_subset[(profiles_subset['depth (dBars)'] < -15) & (profiles_subset['depth (dBars)'] > -25)].groupby('profile_num').mean()
# input_df = input_df.reindex(index=natsorted(input_df.index))

nlags = len(vMag_reshape[:20,100:].mean(axis=0))-1

# var_for_cal = 'chla (ppb)'

#calculate autocorrelations

fig, ax = plt.subplots()
ax2=ax.twinx()

ax2.plot(x[0,100:],vMag_reshape[:20,100:].mean(axis=0), c='red')

ax.plot(x[0,100:],sm.tsa.acf(vMag_reshape[:20,100:].mean(axis=0),nlags=nlags), c='k')


In [ ]:
def crosscorr(datax, datay, lag=0):
    """ Lag-N cross correlation. 
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    return datax.corr(datay.shift(lag))

In [ ]:
profiles_subset

In [ ]:
profiles_subset.chla_clean.shift(3)

In [ ]:
profiles_subset.chla_clean.corr(profiles_subset.ekbackscatter.shift(1))

In [ ]:
plt.scatter(profiles_subset.chla_clean, profiles_subset.ekbackscatter.shift(2),alpha=0.9)

In [ ]:
for p in p_sub.profile_num.unique():
    plt.scatter(p_sub[p_sub.profile_num == p].chla_clean, p_sub[p_sub.profile_num == p].ekbackscatter.shift(25))

In [ ]:
profiles_subset[profiles_subset.profile_num == 't26_25']

In [ ]:
p_sub = profiles_subset[profiles_subset['depth (dBars)'] > -45]

In [ ]:
p_sub = profiles_subset[profiles_subset['depth (dBars)'] > -45]
correls = []
for p in p_sub.profile_num.unique():
    correls.append([crosscorr(p_sub[p_sub.profile_num == p].chla_clean, p_sub[p_sub.profile_num == p].ekbackscatter, lag=i) for i in range(50)])
plt.scatter([list(range(50))]*25,np.array(correls),alpha=0.4)


In [ ]:
plt.scatter([list(range(50))]*25,np.array(correls),alpha=0.4)
# plt.scatter(profiles_subset['depth (dBars)'].values[:50],np.nanmean(np.array(correls),axis=0)+np.nanstd(np.array(correls),axis=0))
# plt.scatter(profiles_subset['depth (dBars)'].values[:50],np.nanmean(np.array(correls),axis=0)-np.nanstd(np.array(correls),axis=0))

In [ ]:
profiles_subset.profile_num.unique()

In [ ]:
plt.scatter(profiles_subset['depth (dBars)'].values[:50],[crosscorr(profiles_subset.chla_clean, profiles_subset.ekbackscatter, lag=i) for i in range(50)])

In [ ]:
[crosscorr(profiles_subset.chla_clean, profiles_subset.ekbackscatter, lag=i) for i in range(50)]

In [ ]:
front_transects, front_locations, time_starts, time_stops

In [ ]:
front_transects

In [ ]:
dfs = []
for i in range(len(time_starts)):
    dfs.append(profs_turb_sub.loc[time_starts[i]:time_stops[i]])
profs_turb_frontal = pd.concat(dfs)

In [ ]:
### calculate the echosounder stuff

In [ ]:
1

In [ ]:
# Create figure with three axes
fig, ax = plt.subplots()

# Plot violin plot on axes 1
parts = ax.violinplot([np.concatenate(shelf_ek),np.concatenate(eddy_ek),np.concatenate(gulf_stream_ek)], positions=[0,1,2], quantiles=[[.25,.50,.75],[.25,.50,.75],[.25,.50,.75]])
# ax1.set_title('Shelf')
ax.set_ylim(92,125)

cs = ['brown', 'yellow', 'blue']

for i,pc in enumerate(parts['bodies']):
    pc.set_facecolor(cs[i])
    pc.set_edgecolor('black')
    pc.set_alpha(1)
    
for partname in ('cbars','cmins','cmaxes', 'cquantiles'):
    vp = parts[partname]
    vp.set_edgecolor('k')
    vp.set_linewidth(1)
    
ax.set_xticks([0,1,2])
ax.set_xticklabels(['Shelf', 'Eddy', 'Gulf Stream'])

ax.set_title("Shelf, Eddy, GS Backscatter Strength")
ax.set_ylabel('Backscatter Strength (sV)')
# plt.savefig('backscatter_strength.png',dpi=300)
plt.show()

np.concatenate(shelf_ek).mean(), np.concatenate(eddy_ek).mean(),np.concatenate(gulf_stream_ek).mean()

# testing some models

In [ ]:
profiles.head(2)

In [ ]:
profs_turb_frontal

In [ ]:
from sklearn.linear_model import LinearRegression

# profiles_sub = profiles[profiles['depth (dBars)'] > -55]
profiles_sub = profs_turb_frontal[profs_turb_frontal['depth (dBars)'] > -80]

chla = profiles_sub['chla (ppb)'].values.copy()
chla[chla > 2] = 2
chla[chla < -0.3] = -0.3
chla = chla + abs(chla.min())

turbidity = profiles_sub['turbidity (FTU)'].values.copy()
turbidity[turbidity > 4] = 4
# turbidity = turbidity + abs(turbidity.min())


# creating an object of LinearRegression class
lr = LinearRegression()
# fitting the training data
X = np.array([profiles_sub['temp (C)'].values, profiles_sub['salinity (PSU)'].values, 
              profiles_sub['depth (dBars)'].values, abs(profiles_sub.datetime.dt.day_of_year.values-225)]).T
y = chla

lr.fit(X, y)

In [ ]:
plt.hist(turbidity,bins=100)
plt.show()

In [ ]:
chla.min()

In [ ]:
plt.hist(chla,bins=400)
plt.show()

In [ ]:
plt.hist(profiles_sub.ekbackscatter,bins=50)
plt.show()

In [ ]:
plt.scatter(chla,profiles_sub.ekbackscatter,alpha=0.01)
plt.show()

In [ ]:
lr.score(X, y)

In [ ]:
lr.coef_

In [ ]:
chla.max()

In [ ]:
plt.scatter(profiles_sub['depth (dBars)'],chla, alpha=0.01)
plt.show()
plt.scatter(profiles_sub['temp (C)'],chla, alpha=0.01)
plt.show()
plt.scatter(profiles_sub['salinity (PSU)'],chla, alpha=0.01)
plt.show()
plt.scatter(profiles_sub['potential density (kg/m^3 -1000)'],chla, alpha=0.01)
plt.show()
plt.scatter(turbidity,chla, alpha=0.01)
plt.show()
plt.scatter(abs(profiles_sub.datetime.dt.day_of_year.values-225),chla, alpha=0.01)
plt.show()



In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.scatter(chla, lr.predict(X), alpha=0.05)
ax.set_xlim(0,2)
ax.set_ylim(0,2)
ax.set_ylabel('predicted')
ax.set_xlabel('measured')

In [ ]:
plt.scatter(abs(profiles_sub.datetime.dt.day_of_year.values-225), profiles_sub['temp (C)'])
plt.xlim(0,225)

In [ ]:
plt.scatter(profiles_sub['chla_clean'], profiles_sub['ekbackscatter'],alpha=0.3)
# plt.xlim(0,225)

In [ ]:
plt.scatter(turbidity, chla,alpha=0.3)
# plt.xlim(0,225)

In [ ]:
plt.hist(turbidity)
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 15})

In [ ]:
fig = plt.figure(figsize=(18,18))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(profiles_sub['temp (C)'], profiles_sub['salinity (PSU)'], profiles_sub['depth (dBars)'],c=chla, cmap='viridis',alpha=0.2,vmin=.0,vmax=1.5)
ax.set_ylabel('Sal (PSU)')
ax.set_xlabel('Temp (C)')
ax.set_zlabel('Depth')
fig.savefig('chla_across_TS_d.png',dpi=300)

In [ ]:
fig = plt.figure(figsize=(18,18))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(profiles_sub['temp (C)'], profiles_sub['salinity (PSU)'], abs(profiles_sub.datetime.dt.day_of_year.values-225),c=chla, cmap='viridis',alpha=0.2,vmin=.0,vmax=1.)

ax.set_ylabel('Sal (PSU)')
ax.set_xlabel('Temp (C)')
ax.set_zlabel('Days from Aug 15')
fig.savefig('chla_across_TS_DOY.png',dpi=300)

In [ ]:
fig = plt.figure(figsize=(18,18))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(abs(profiles_sub.datetime.dt.day_of_year.values-225), profiles_sub['temp (C)'], profiles_sub['depth (dBars)'],c=chla, cmap='viridis',alpha=0.5,vmin=.0,vmax=1.)

ax.set_xlabel('Days from Aug 15')
ax.set_ylabel('Temp (C)')
ax.set_zlabel('Depth')
fig.savefig('chla_across_t_s_d.png',dpi=300)

In [ ]:
fig = plt.figure(figsize=(18,18))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(abs(profiles_sub.datetime.dt.day_of_year.values-225), profiles_sub['temp (C)'], profiles_sub['depth (dBars)'],c=profiles_sub['ekbackscatter'], cmap='jet',alpha=0.5,vmin=101,vmax=109)

ax.set_xlabel('Days from Aug 15')
ax.set_ylabel('Temp (C)')
ax.set_zlabel('Depth')
# fig.savefig('eksv_across_t_s_d.png',dpi=300)

In [ ]:
fig = plt.figure(figsize=(18,18))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(profiles_sub['temp (C)'], profiles_sub['salinity (PSU)'], abs(profiles_sub.datetime.dt.day_of_year.values-225),c=profiles_sub['ekbackscatter'], cmap='jet',alpha=0.5,vmin=101,vmax=109)

ax.set_ylabel('Sal (PSU)')
ax.set_xlabel('Temp (C)')
ax.set_zlabel('Days from Aug 15')
# fig.savefig('eksv_across_TS_DOY.png',dpi=300)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,8))
ax[0].scatter(profiles_sub['ekbackscatter'][profiles_sub['salinity (PSU)']>36], chla[profiles_sub['salinity (PSU)'] > 36] ,c= abs(profiles_sub.datetime.dt.day_of_year.values-225)[profiles_sub['salinity (PSU)']>36],alpha=.1)# cmap='jet',alpha=0.5,vmin=101,vmax=109)
ax[0].set_xlim(93,125)

ax[1].scatter(profiles_sub['ekbackscatter'][profiles_sub['salinity (PSU)']<36], chla[profiles_sub['salinity (PSU)'] < 36] ,c= abs(profiles_sub.datetime.dt.day_of_year.values-225)[profiles_sub['salinity (PSU)']<36],alpha=.1)# cmap='jet',alpha=0.5,vmin=101,vmax=109)
ax[1].set_xlim(93,125)

In [ ]:
fig = plt.figure(figsize=(18,18))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(profiles_sub['ekbackscatter'], chla , profiles_sub['salinity (PSU)'],c=abs(profiles_sub.datetime.dt.day_of_year.values-225))# cmap='jet',alpha=0.5,vmin=101,vmax=109)

ax.set_ylabel('chla')
ax.set_xlabel('acoustic backscatter')
ax.set_zlabel('Days from Aug 15')
# fig.savefig('eksv_across_TS_DOY.png',dpi=300)

In [ ]:
fig = plt.figure(figsize=(18,18))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(profiles_sub['temp (C)'], profiles_sub['salinity (PSU)'], profiles_sub['depth (dBars)'],c=profiles_sub['ekbackscatter'], cmap='jet',alpha=0.5,vmin=101,vmax=110)

ax.set_ylabel('Sal (PSU)')
ax.set_xlabel('Temp (C)')
ax.set_zlabel('Depth')
# fig.savefig('chla_across_TS_d.png',dpi=300)

In [ ]:
fig = plt.figure(figsize=(18,18))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(abs(profiles_sub.datetime.dt.day_of_year.values-225), profiles_sub['temp (C)'], profiles_sub['depth (dBars)'],c=turbidity, cmap=cmocean.cm.turbid,alpha=0.5,vmin=.95,vmax=1.7)

ax.set_xlabel('Days from Aug 15')
ax.set_ylabel('Temp (C)')
ax.set_zlabel('Depth')
# fig.savefig('turb_across_t_s_d.png',dpi=300)

In [ ]:
fig = plt.figure(figsize=(18,18))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(profiles_sub['temp (C)'], profiles_sub['salinity (PSU)'], abs(profiles_sub.datetime.dt.day_of_year.values-225),c=profiles_sub['turbidity (FTU)'], cmap=cmocean.cm.turbid,alpha=0.5,vmin=.95,vmax=1.7)

ax.set_ylabel('Sal (PSU)')
ax.set_xlabel('Temp (C)')
ax.set_zlabel('Days from Aug 15')
# fig.savefig('turb_across_t_s_d.png',dpi=300)

In [ ]:
! pip install pygam

In [ ]:
X

In [ ]:
from pygam import PoissonGAM, s, te
from pygam.datasets import chicago

# X, y = chicago(return_X_y=True)

gam = PoissonGAM(s(0, n_splines=10) + te(2, 3) + s(1)).fit(X, y)
# gam = PoissonGAM(s(0) + s(1) + te(0, 3,n_splines=6)).fit(X, y)

T, S, depth, DOY

In [ ]:
XX = gam.generate_X_grid(term=2, meshgrid=True)
XX

In [ ]:
from mpl_toolkits import mplot3d

plt.ion()
plt.rcParams['figure.figsize'] = (12, 8)

XX = gam.generate_X_grid(term=1, meshgrid=True)
Z = gam.partial_dependence(term=1, X=XX, meshgrid=True)

ax = plt.axes(projection='3d')
ax.plot_surface(XX[0], XX[1], Z, cmap='viridis')
ax.set_xlabel('Depth')
ax.set_ylabel('DOY Dist from Summer')
ax.set_zlabel('Chla Impact')

In [ ]:
from pygam import PoissonGAM, s, te
from pygam.datasets import chicago

# X, y = chicago(return_X_y=True)

# gam = PoissonGAM(s(0, n_splines=10) + te(2, 3) + s(1)).fit(X, y)
gam = PoissonGAM(s(0,n_splines=5) + s(1,n_splines=5) + te(0, 3,n_splines=5)).fit(X, y)

In [ ]:
XX[0].shape

In [ ]:
from mpl_toolkits import mplot3d

plt.ion()
plt.rcParams['figure.figsize'] = (12, 8)

XX = gam.generate_X_grid(term=2, meshgrid=True)
Z = gam.partial_dependence(term=2, X=XX, meshgrid=True)
# ax.plot(XX, gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')

ax = plt.axes(projection='3d')
ax.plot_surface(XX[0], XX[1], Z, cmap='viridis')
ax.set_xlabel('T')
ax.set_ylabel('DOY Dist from Summer')
ax.set_zlabel('Chla Impact')

In [ ]:
titles = ['T', 'S']

fig, axs = plt.subplots(1,len(titles),figsize=(20,6));
for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
#     ax.set_ylim(-2,2)
    ax.set_title(titles[i]);

In [ ]:
from pygam import PoissonGAM, s, te, LinearGAM
from pygam.datasets import chicago

# X, y = chicago(return_X_y=True)

gam = LinearGAM(s(0,n_splines=6) + s(1,n_splines=6) + s(2,n_splines=6) + s(3,n_splines=6)).fit(X, y)
# gam = PoissonGAM(s(0) + s(1) + te(0, 1)).fit(X[:,3:], y)
gam.summary()


In [ ]:
titles = ['T', 'S', 'depth', 'DOY']

fig, axs = plt.subplots(1,len(titles),figsize=(20,6));
for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
#     ax.set_ylim(-2,2)
    ax.set_title(titles[i]);
#     break

### Make a PCA plot with the variable vectors plotted

In [ ]:
profiles[['depth (dBars)','temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla (ppb)','turbidity (FTU)','ekbackscatter','daysfromaug15']]

In [ ]:
chla.shape

In [ ]:
len(profiles[profiles['turbidity (FTU)']>3])

In [ ]:
profiles[profiles['turbidity (FTU)']>3]

In [ ]:
plt.hist(profiles[(profiles['turbidity (FTU)']<5) & (profiles['turbidity (FTU)']>2.5)]['turbidity (FTU)'],bins=500)
plt.show()

In [ ]:
chla = profiles['chla (ppb)'].values.copy()
chla[chla > 2] = 2
chla[chla < -0.3] = -0.3
chla = chla + abs(chla.min())

profiles['chla_clean'] = chla

###

turbidity = profiles['turbidity (FTU)'].values.copy()
turbidity[turbidity > 4] = 4

profiles['turbidity_clean'] = turbidity

In [ ]:
profs_turb_sub = profiles[profiles['turbidity (FTU)']<3]

In [ ]:
profiles[profiles['chla (ppb)'] > 2]['chla (ppb)']

In [ ]:
plt.hist(turbidity)

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
pd.plotting.scatter_matrix(profiles_sub[['depth (dBars)','temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla_clean','turbidity_clean','daysfromaug15', 'cluster']], alpha=0.05,ax=ax)
plt.show()

In [ ]:
plt.hist(profiles['turbidity_clean'])

In [ ]:
profiles.head(3)

In [ ]:
(profiles['salinity (PSU)'] > profiles['gs_sal_thresh']).astype(int)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = profiles_sub[['temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla_clean','turbidity_clean']]

#In general it is a good idea to scale the data
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

pca = PCA()
pca.fit(X)
x_new = pca.transform(X)   

labels = ['temp','salinity','potential density','chla','turbidity','daysfromaug15']

def myplot(score,coeff,ax,labels,j):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]

#     plt.scatter(xs ,ys, c = y,cmap='jet',alpha=.01,vmin=-80,vmax=0)#vmin=1.,vmax=1.5,alpha=0.1) #without scaling
    if j == 0:
#         im = ax.scatter(xs ,ys, color='darkgrey',alpha=0.01)#vmin=1.,vmax=1.5,alpha=0.1) #without scaling
#         im = ax.scatter(xs ,ys, c = (profiles['salinity (PSU)'] > profiles['gs_sal_thresh']).astype(int),cmap='bwr',alpha=.01) #without scaling
        im = ax.scatter(xs ,ys, c=profiles_sub['salinity (PSU)'],cmap=cmocean.cm.haline,alpha=.1,vmin=35,vmax=36.5) #without scaling
        ax.set_title('Sal')
    if j == 1:
        im = ax.scatter(xs ,ys, c=profiles_sub['temp (C)'],cmap=cmocean.cm.thermal,alpha=.1) #without scaling
        ax.set_title('Temp')
        
    if j == 2:
        im = ax.scatter(xs ,ys, c = profiles_sub['potential density (kg/m^3 -1000)'],cmap=cmocean.cm.dense,alpha=.2,vmin=21.2,vmax=27) #without scaling
        ax.set_title('Density')
        
    elif j == 3:
        im = ax.scatter(xs ,ys, c=profiles_sub['depth (dBars)'],cmap='cividis_r',alpha=.1,vmin=-80,vmax=0) #without scaling
        ax.set_title('Depth')
        
    elif j == 4:
        im = ax.scatter(xs ,ys, c=profiles_sub['daysfromaug15'],cmap=cmocean.cm.solar_r,alpha=.1,vmin=0,vmax=225) #without scaling
        ax.set_title('Days from Aug 15')
        
    elif j == 5:
        im = ax.scatter(xs ,ys, c = profiles_sub['turbidity_clean'],cmap=cmocean.cm.turbid,alpha=.2,vmin=1.,vmax=1.8) #without scaling
        ax.set_title('Turbidity')
    elif j == 6:
        im = ax.scatter(xs ,ys, c = profiles_sub['chla_clean'],cmap=cmocean.cm.algae,alpha=.2,vmin=0.,vmax=1.5) #without scaling
        ax.set_title('Chla Fluorescence')
    elif j == 7:
        im = ax.scatter(xs ,ys, c = profiles_sub['ekbackscatter'],cmap='jet',alpha=.2,vmin=98,vmax=112) #without scaling
        ax.set_title('EK Backscatter')
    elif j == 8:
#         im = ax.scatter(xs ,ys, c = (profiles['salinity (PSU)'] > profiles['gs_sal_thresh']).astype(int),cmap='bwr',alpha=.02) #without scaling
          
#         ax.set_title('GS (red) vs Coastal (blue)')
    
        im = ax.scatter(xs ,ys, c = profiles_sub.cluster  ,cmap='jet',alpha=.1,vmin=0,vmax=6) #without scaling
        
    fig.colorbar(im,ax=ax)
    
    for i in range(n):
        ax.arrow(0, 0, coeff[i,0]*3, coeff[i,1]*3,color = 'r',alpha = 0.5, lw=2, head_width=.25,head_length=.25)
        if labels is None:
            ax.text(coeff[i,0]* 6, coeff[i,1] * 6, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            ax.text(coeff[i,0]* 4, coeff[i,1] * 4, labels[i], color = 'black', ha = 'center', va = 'center')

fig, ax = plt.subplots(3,3,figsize=(26,22))

for i,ax in enumerate([ax[0,0],ax[0,1],ax[0,2],ax[1,0],ax[1,1],ax[1,2],ax[2,0],ax[2,1],ax[2,2]]):
    ax.set_xlabel("PC{}".format(1))
    ax.set_ylabel("PC{}".format(2))
    ax.grid()
    ax.set_xlim(-3,6.2)
    ax.set_ylim(-3,6.5)

    #Call the function. 
    myplot(x_new[:,0:2], pca.components_.T,ax=ax, labels=labels,j=i) 
fig.savefig('pca_profiles.png',dpi=300)
plt.show()

In [ ]:
from sklearn.cluster import OPTICS, cluster_optics_dbscan


In [ ]:

clust = OPTICS(min_samples=50, xi=0.05, min_cluster_size=0.05)

# Run the fit
clust.fit(X)

In [ ]:
np.unique(clust.labels_,return_counts=True)

In [ ]:

labels_200 = cluster_optics_dbscan(
    reachability=clust.reachability_,
    core_distances=clust.core_distances_,
    ordering=clust.ordering_,
    eps=.1,
)

In [ ]:

np.unique(labels_200,return_counts=True)

In [ ]:
from sklearn.cluster import KMeans

Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(X)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
X.shape

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 10,15]
from sklearn.metrics import silhouette_score
silhouette_avg = []

for num_clusters in range_n_clusters:
    # initialise kmeans
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(X)
    cluster_labels = kmeans.labels_
    # silhouette score
    silhouette_avg.append(silhouette_score(X, cluster_labels,sample_size=10000))
    print(num_clusters)

In [ ]:
plt.plot(range_n_clusters,silhouette_avg,'bx-')
plt.xlabel('Values of K') 
plt.ylabel('Silhouette score') 
plt.title('Silhouette analysis For Optimal k')
plt.show()

In [ ]:
X = profs_turb_sub[['temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla_clean','turbidity_clean']]

#In general it is a good idea to scale the data
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

kmeans = KMeans(n_clusters=7)
kmeans.fit(X)
cluster_labels = kmeans.labels_

In [ ]:
kmeans.labels_.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = profiles_gs[['depth (dBars)','temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla_clean','turbidity_clean','daysfromaug15']]

#In general it is a good idea to scale the data
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

pca = PCA()
pca.fit(X)
x_new = pca.transform(X)   

labels = ['depth (dBars)','temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla (ppb)','turbidity (FTU)','daysfromaug15']

def myplot(score,coeff,ax,labels,j):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]

#     plt.scatter(xs ,ys, c = y,cmap='jet',alpha=.01,vmin=-80,vmax=0)#vmin=1.,vmax=1.5,alpha=0.1) #without scaling
    if j == 0:
        im = ax.scatter(xs ,ys, color='blue',alpha=0.02)#vmin=1.,vmax=1.5,alpha=0.1) #without scaling
    elif j == 1:
        im = ax.scatter(xs ,ys, c = profiles_gs['chla_clean'],cmap=cmocean.cm.algae,alpha=.2,vmin=0.,vmax=1.5) #without scaling
    elif j == 2:
        im = ax.scatter(xs ,ys, c = profiles_gs['ekbackscatter'],cmap='jet',alpha=.2,vmin=100.,vmax=112) #without scaling
    elif j == 3:
        im = ax.scatter(xs ,ys, c = profiles_gs['turbidity_clean'],cmap=cmocean.cm.turbid,alpha=.2,vmin=1.,vmax=1.8) #without scaling
    
    fig.colorbar(im,ax=ax)
    
    for i in range(n):
        ax.arrow(0, 0, coeff[i,0]*5, coeff[i,1]*5,color = 'r',alpha = 0.5, lw=2, head_width=.25,head_length=.25)
        if labels is None:
            ax.text(coeff[i,0]* 6, coeff[i,1] * 6, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            ax.text(coeff[i,0]* 6.5, coeff[i,1] * 7, labels[i], color = 'black', ha = 'center', va = 'center')

fig, ax = plt.subplots(2,2,figsize=(18,14))

for i,ax in enumerate([ax[0,0],ax[0,1],ax[1,0],ax[1,1]]):
    ax.set_xlabel("PC{}".format(1))
    ax.set_ylabel("PC{}".format(2))
    ax.grid()
    ax.set_xlim(-4,5)
    ax.set_ylim(-4,4)

    #Call the function. 
    myplot(x_new[:,0:2], pca.components_.T,ax=ax, labels=labels,j=i) 
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

profiles_ek = profiles[profiles['ekbackscatter'].notna()]

X = profiles_ek[['depth (dBars)','temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla_clean','turbidity_clean','daysfromaug15', 'ekbackscatter']]

#In general it is a good idea to scale the data
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

pca = PCA()
pca.fit(X)
x_new = pca.transform(X)   

labels = ['depth','temp','salinity','potential density','chla','turbidity','daysfromaug15', 'ekbackscatter']

def myplot(score,coeff,ax,labels,j):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]

#     plt.scatter(xs ,ys, c = y,cmap='jet',alpha=.01,vmin=-80,vmax=0)#vmin=1.,vmax=1.5,alpha=0.1) #without scaling
    if j == 0:
#         im = ax.scatter(xs ,ys, color='darkgrey',alpha=0.01)#vmin=1.,vmax=1.5,alpha=0.1) #without scaling
#         im = ax.scatter(xs ,ys, c = (profiles['salinity (PSU)'] > profiles['gs_sal_thresh']).astype(int),cmap='bwr',alpha=.01) #without scaling
        im = ax.scatter(xs ,ys, c=profiles_ek['salinity (PSU)'],cmap=cmocean.cm.haline,alpha=.1,vmin=35,vmax=36.5) #without scaling
        ax.set_title('Sal')
    if j == 1:
        im = ax.scatter(xs ,ys, c=profiles_ek['daysfromaug15'],cmap='cividis_r',alpha=.1,vmin=0,vmax=225) #without scaling
        ax.set_title('Days from Aug 15')
    if j == 2:
        im = ax.scatter(xs ,ys, c=profiles_ek['temp (C)'],cmap=cmocean.cm.thermal,alpha=.1) #without scaling
        ax.set_title('Temp')
    elif j == 3:
        im = ax.scatter(xs ,ys, c = profiles_ek['chla_clean'],cmap=cmocean.cm.algae,alpha=.2,vmin=0.,vmax=1.5) #without scaling
        ax.set_title('Chla Fluorescence')
    elif j == 4:
        im = ax.scatter(xs ,ys, c = profiles_ek['turbidity_clean'],cmap=cmocean.cm.turbid,alpha=.2,vmin=1.,vmax=1.8) #without scaling
        ax.set_title('Turbidity')
    elif j == 5:
        im = ax.scatter(xs ,ys, c = profiles_ek['ekbackscatter'],cmap='jet',alpha=.2,vmin=100.,vmax=112) #without scaling
        ax.set_title('EK Backscatter')
    
    fig.colorbar(im,ax=ax)
    
    for i in range(n):
        ax.arrow(0, 0, coeff[i,0]*5, coeff[i,1]*5,color = 'r',alpha = 0.5, lw=2, head_width=.25,head_length=.25)
        if labels is None:
            ax.text(coeff[i,0]* 6, coeff[i,1] * 6, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            ax.text(coeff[i,0]* 6.5, coeff[i,1] * 6.5, labels[i], color = 'black', ha = 'center', va = 'center')

fig, ax = plt.subplots(3,2,figsize=(18,22))

for i,ax in enumerate([ax[0,0],ax[0,1],ax[1,0],ax[1,1],ax[2,0],ax[2,1]]):
    ax.set_xlabel("PC{}".format(1))
    ax.set_ylabel("PC{}".format(2))
    ax.grid()
    ax.set_xlim(-4,5)
    ax.set_ylim(-3,5)

    #Call the function. 
    myplot(x_new[:,0:2], pca.components_.T,ax=ax, labels=labels,j=i) 
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = profiles_coastal[['depth (dBars)','temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla_clean','turbidity_clean','daysfromaug15']]

#In general it is a good idea to scale the data
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

pca = PCA()
pca.fit(X)
x_new = pca.transform(X)   

labels = ['depth (dBars)','temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla (ppb)','turbidity (FTU)','daysfromaug15']

def myplot(score,coeff,ax,labels,j):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]

#     plt.scatter(xs ,ys, c = y,cmap='jet',alpha=.01,vmin=-80,vmax=0)#vmin=1.,vmax=1.5,alpha=0.1) #without scaling
    if j == 0:
        im = ax.scatter(xs ,ys, color='blue',alpha=0.02)#vmin=1.,vmax=1.5,alpha=0.1) #without scaling
    elif j == 1:
        im = ax.scatter(xs ,ys, c = profiles_coastal['chla_clean'],cmap=cmocean.cm.algae,alpha=.2,vmin=0.,vmax=1.5) #without scaling
    elif j == 2:
        im = ax.scatter(xs ,ys, c = profiles_coastal['ekbackscatter'],cmap='jet',alpha=.2,vmin=100.,vmax=112) #without scaling
    elif j == 3:
        im = ax.scatter(xs ,ys, c = profiles_coastal['turbidity_clean'],cmap=cmocean.cm.turbid,alpha=.2,vmin=1.,vmax=1.8) #without scaling
    
    fig.colorbar(im,ax=ax)
    
    for i in range(n):
        ax.arrow(0, 0, coeff[i,0]*5, coeff[i,1]*5,color = 'r',alpha = 0.5, lw=2, head_width=.25,head_length=.25)
        if labels is None:
            ax.text(coeff[i,0]* 6, coeff[i,1] * 6, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            ax.text(coeff[i,0]* 6.5, coeff[i,1] * 7, labels[i], color = 'black', ha = 'center', va = 'center')

fig, ax = plt.subplots(2,2,figsize=(18,14))

for i,ax in enumerate([ax[0,0],ax[0,1],ax[1,0],ax[1,1]]):
    ax.set_xlabel("PC{}".format(1))
    ax.set_ylabel("PC{}".format(2))
    ax.grid()
    ax.set_xlim(-4,5)
    ax.set_ylim(-4,4)

    #Call the function. 
    myplot(x_new[:,0:2], pca.components_.T,ax=ax, labels=labels,j=i) 
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = profiles[['depth (dBars)','temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla_clean','turbidity_clean','daysfromaug15']]

y = profiles['chla_clean']

#In general it is a good idea to scale the data
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

pca = PCA()
pca.fit(X)
x_new = pca.transform(X)   

labels = ['depth (dBars)','temp (C)','salinity (PSU)','potential density (kg/m^3 -1000)','chla (ppb)','turbidity (FTU)','daysfromaug15']

def myplot(score,coeff,labels=labels):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]

    plt.scatter(xs ,ys, c = y,cmap=cmocean.cm.algae,alpha=.2,vmin=0.,vmax=1.5) #without scaling
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0]*5, coeff[i,1]*5,color = 'r',alpha = 0.5)
        if labels is None:
            plt.text(coeff[i,0]* 5, coeff[i,1] * 5, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            plt.text(coeff[i,0]* 5, coeff[i,1] * 5, labels[i], color = 'black', ha = 'center', va = 'center')

plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))
plt.grid()
plt.ylim(-4,4)

#Call the function. 
myplot(x_new[:,0:2], pca.components_.T) 
plt.show()

In [ ]:
plt.scatter(profiles['depth (dBars)'],profiles['temp (C)'],alpha=0.1)

In [ ]:
pca.components_.T.shape

In [ ]:
single_front_locations = [f[0] for f in front_locations]

now plot all the transects where the calculated front is 0 and the hot side is - and the cold side is +

In [ ]:
dist_list = []
front_location_list = []
sst_list = []
current_list = []
echo_list = []
interp_sst_list = []
interp_current_list = []
interp_echo_list = []

sal_front_list = []

vmp_sal_list = []
vmp_temp_list = []
vmp_chla_list = []
vmp_dt_list = []

grid_spacing = 20

counter = 0

for idx,fn in enumerate(filenames):
    if idx in [0,1,2,3]:
        continue
    if idx not in front_transects:
        continue
    try:
        ##########
        
        dist = scipy.io.loadmat(fn+'_distance.mat')['distance']
        sst = scipy.io.loadmat(fn+'_SST.mat')['SST']
        time = scipy.io.loadmat(fn+'_time.mat')['time']
        timestamps = pd.to_datetime(time[0,:]-719529, unit='D')
       
        
#         if idx in [5,28,39,40,41]:
#             dist = (dist - dist[-1])*-1
        
        X = scipy.io.loadmat(fn+'_X.mat')['X']
        Y = scipy.io.loadmat(fn+'_Y.mat')['Y']
        vMag = scipy.io.loadmat(fn+'_vmag.mat')['C']

        vDir = scipy.io.loadmat(fn+'_vdir.mat')['C']
        
        vShear = scipy.io.loadmat(fn+'_vshear.mat')['C']
        
        echo = scipy.io.loadmat(fn+'_echo.mat')['C']
        Xecho = scipy.io.loadmat(fn+'_Xecho.mat')['X']
        Yecho = scipy.io.loadmat(fn+'_Yecho.mat')['Y']
        
        # correcting for the smaller bin size
        Yecho = (Yecho*0.502)-1
        
        lat = scipy.io.loadmat(fn+'_lat.mat')['lat']
        long = scipy.io.loadmat(fn+'_long.mat')['long']
        
        # the distances are crazy wrong sometimes so re-calcuating the distance
        coords_1 = (lat[0], long[0])
        distances = []
        for i in range(len(lat)):
            coords_2 = (lat[i], long[i])
            distances.append(distance.geodesic(coords_1, coords_2).meters)
        dist = np.array(distances)       
#         bogus_dist_locations = dist > 35000        
#         dist[dist > 25000] = np.nan
#         dist = pd.Series(dist).interpolate().values
        # checking if the current point is within 500 of one that was good, assuming the first point is good
        # then if not setting it to nan and interpolating it out
        last_good_idx = 0
        bad_idxs = []
        for di,d in enumerate(dist):
            if dist[di] - dist[last_good_idx] < 500:
                last_good_idx = di
            else:
                bad_idxs.append(di)
        dist[bad_idxs] = np.nan
        dist = pd.Series(dist).interpolate().values

        dist = dist.reshape(-1,1)
        
        adcp_df = pd.DataFrame({'dt':timestamps,
                    'datetime':timestamps,
                    'sst':sst[:,0],
                    'lat': lat[:,0],
                    'lon':long[:,0],
                    'dist':dist[:,0]})

        adcp_df = adcp_df.set_index('dt')
        adcp_df = adcp_df.sort_index(ascending=True)
        
        
        ##########
        
        sal_vmp, temp_vmp, chla_vmp, turb_vmp, dt_vmp, dist_vmp = grab_vmp_data(profiles, adcp_df, timestamps[0],timestamps[-1])
            
        vmp_sal_list.append(sal_vmp)
        vmp_temp_list.append(temp_vmp)
        vmp_chla_list.append(chla_vmp)
        vmp_dt_list.append(dt_vmp)
        
        ################

        
        
        if idx in [5,28,39,40,41]:
            dist = (dist - dist[-1])*-1
        

        vMag_reshape = vMag.reshape(-1,69).T
        vDir_reshape = vDir.reshape(-1,69).T
        echo_reshape = echo.reshape(-1,127).T
        echo_top_layer = np.mean(echo_reshape[0:8,:],axis=0)
#         x, y = np.meshgrid(Xecho.reshape(-1,127)[:,0],Yecho[:127,0])    



#         x, y = np.meshgrid(X.reshape(-1,69)[:,0],Y[:69,0])    

        N=180
        current_speed = np.mean(vMag_reshape[0:4,:],axis=0)
        current_speed_smooth = pd.Series(current_speed).rolling(window=N).mean().iloc[N-1:].values
        vgradient = []
        step = 1

        for i in range(len(current_speed)-N):
            du = current_speed_smooth[i] - current_speed_smooth[i+step]
            dx = dist[i] - dist[i+step]
            current_grad = du/dx
            vgradient.append(current_grad)

        dudx = scipy.signal.savgol_filter(current_speed_smooth, window_length=11, polyorder=2, deriv=1)
        # taking off the first 100 points because those are often noisy and never the front
        front_location = np.argmax(np.abs(dudx[100:-30]))+100
        
        current_list.append(current_speed)
        sst_list.append(sst)
        dist_list.append(dist)
        echo_list.append(echo_top_layer)
        
#         dist_centered = dist - dist[front_location]
        dist_centered = dist - single_front_locations[counter]
        counter += 1
        
        sst_reg = np.interp(np.arange(-12000,12000, grid_spacing), dist_centered[:,0], sst[:,0], left=np.nan, right=np.nan, period=None)
        interp_sst_list.append(sst_reg)
        
        current_reg = np.interp(np.arange(-12000,12000, grid_spacing), dist_centered[:,0], current_speed, left=np.nan, right=np.nan, period=None)
        interp_current_list.append(current_reg)
        
        dist_centered_echo = dist_centered[:,0]
        while len(dist_centered_echo) != len(echo_top_layer):
            
            if len(dist_centered_echo) > len(echo_top_layer):
                dist_centered_echo = dist_centered_echo[:-1]
                
            elif len(dist_centered[:,0]) < len(echo_top_layer):
                echo_top_layer = echo_top_layer[:-1]

        
        echo_reg = np.interp(np.arange(-12000,12000, grid_spacing), dist_centered_echo, echo_top_layer, left=np.nan, right=np.nan, period=None)
        interp_echo_list.append(echo_reg)
        
#         print('did one')
    except Exception as e: 
        print(e)



In [ ]:
1

In [ ]:
sal_est = provide_gs_sal_threshold(np.concatenate(vmp_dt_list), buffer=0.25)
gs_water = np.concatenate(vmp_sal_list) > sal_est


temp_est = provide_gs_temp_threshold(np.concatenate(vmp_dt_list), buffer=0.25)
gs_water_temp = np.concatenate(vmp_temp_list) > temp_est

In [ ]:
tester = []
for i in range(len(gs_water)):
    if gs_water[i] and not gs_water_temp[i]:
        tester.append(True)
    else:
        tester.append(False)

In [ ]:
tester = []
for i in range(len(gs_water)):
    if not gs_water[i] or not gs_water_temp[i]:
        tester.append(False)
    else:
        tester.append(True)

I want all the coastal water from both of them

In [ ]:
(temp_est-annual_temp_mean).shape

In [ ]:
plt.hist(profiles.datetime.dt.day_of_year,bins=52)
plt.show()

In [ ]:
# Figure out boudaries (mins and maxs)
smin = 32-.7
smax = 37+.7
tmin = 8-.7 
tmax = 30+.7

# Calculate how many gridcells we need in the x and y dimensions
xdim = int(round((smax-smin)/0.1+1,0))
ydim = int(round((tmax-tmin)+1,0))
 
# Create empty grid of zeros
dens = np.zeros((ydim,xdim))
 
# Create temp and salt vectors of appropiate dimensions
ti = np.linspace(1,ydim-1,ydim)+tmin
si = np.linspace(1,xdim-1,xdim)*0.1+smin
 
# Loop to fill in grid with densities
for j in range(0,int(ydim)):
    for i in range(0, int(xdim)):
        dens[j,i]=gsw.rho(si[i],ti[j],0)

# Substract 1000 to convert to sigma-t
dens = dens - 1000
    
# plot T and S diagrams first with distance from coast and then with other variables
fig, ax = plt.subplots(figsize=(8.5,7))
markersize=40

ax.set_xlim(32,37)
ax.set_ylim(8,30)
ax.set_xlabel('Salinity')
ax.set_ylabel('Temperature (C)')

CS = ax.contour(si,ti,dens, linestyles='dashed', colors='k', alpha=0.5)
ax.clabel(CS, fontsize=12, inline=1, fmt='%1.1f') # Label every second level
# ellipse = Ellipse(xy=(35.88, 28.05), width=0.5, height=1.2, angle=-20,
#                 edgecolor='k', fc='None', ls='--', lw=2)
# ax.add_patch(ellipse)

# im = ax.scatter(np.concatenate(vmp_sal_list), np.concatenate(vmp_temp_list), c=np.concatenate(vmp_chla_list), vmin=-0.5,vmax=0.5,alpha=0.4)#c=np.concatenate(vmp_dt_list), vmin=0,vmax=365, cmap='twilight')

temp_est = provide_gs_temp_threshold(np.concatenate(vmp_dt_list))
sal_est = provide_gs_sal_threshold(np.concatenate(vmp_dt_list))

im = ax.scatter(profiles[profiles['depth (dBars)'] < -35]['salinity (PSU)'], profiles[profiles['depth (dBars)'] < -35]['temp (C)'], c=profiles[profiles['depth (dBars)'] < -35].datetime.dt.day_of_year, cmap=cmocean.cm.phase, alpha=0.1)


fig.colorbar(im)
plt.show()

# plot T and S diagrams first with distance from coast and then with other variables
fig, ax = plt.subplots(figsize=(8.5,7))
markersize=40

ax.set_xlim(32,37)
ax.set_ylim(8,30)
ax.set_xlabel('Salinity')
ax.set_ylabel('Temperature (C)')

CS = ax.contour(si,ti,dens, linestyles='dashed', colors='k', alpha=0.5)
ax.clabel(CS, fontsize=12, inline=1, fmt='%1.1f') # Label every second level
# ellipse = Ellipse(xy=(35.88, 28.05), width=0.5, height=1.2, angle=-20,
#                 edgecolor='k', fc='None', ls='--', lw=2)
# ax.add_patch(ellipse)

# im = ax.scatter(np.concatenate(vmp_sal_list), np.concatenate(vmp_temp_list), c=np.concatenate(vmp_chla_list), vmin=-0.5,vmax=0.5,alpha=0.4)#c=np.concatenate(vmp_dt_list), vmin=0,vmax=365, cmap='twilight')

temp_est = provide_gs_temp_threshold(np.concatenate(vmp_dt_list))
sal_est = provide_gs_sal_threshold(np.concatenate(vmp_dt_list))

im = ax.scatter(profiles['salinity (PSU)'], profiles['temp (C)'], c=profiles['depth (dBars)'], cmap=cmocean.cm.deep_r, alpha=0.25, vmin=-100,vmax=0)


fig.colorbar(im)
plt.show()

# plot T and S diagrams first with distance from coast and then with other variables
fig, ax = plt.subplots(figsize=(8.5,7))
markersize=40

# ax.set_xlim(32,37)
# ax.set_ylim(8,30)
ax.set_xlabel('Salinity')
ax.set_ylabel('Temperature (C)')

CS = ax.contour(si,ti,dens, linestyles='dashed', colors='k', alpha=0.5)
ax.clabel(CS, fontsize=12, inline=1, fmt='%1.1f') # Label every second level
# ellipse = Ellipse(xy=(35.88, 28.05), width=0.5, height=1.2, angle=-20,
#                 edgecolor='k', fc='None', ls='--', lw=2)
# ax.add_patch(ellipse)

# im = ax.scatter(np.concatenate(vmp_sal_list), np.concatenate(vmp_temp_list), c=np.concatenate(vmp_chla_list), vmin=-0.5,vmax=0.5,alpha=0.4)#c=np.concatenate(vmp_dt_list), vmin=0,vmax=365, cmap='twilight')

temp_est = provide_gs_temp_threshold(np.concatenate(vmp_dt_list))
sal_est = provide_gs_sal_threshold(np.concatenate(vmp_dt_list))

depth = -50

im = ax.scatter(profiles[profiles['depth (dBars)'] < depth]['salinity (PSU)'], profiles[profiles['depth (dBars)'] < depth]['temp (C)'], c=profiles[profiles['depth (dBars)'] < depth]['depth (dBars)'], cmap=cmocean.cm.deep_r, alpha=0.25, vmin=-100,vmax=depth)


fig.colorbar(im)
plt.show()

In [ ]:
markers = []
for i in gs_water.astype(np.int):
    if i == 0:
        markers.append('^')
    else:
        markers.append('o')

In [ ]:
# Figure out boudaries (mins and maxs)
smin = 32-.7
smax = 37+.7
tmin = 8-.7 
tmax = 30+.7

# Calculate how many gridcells we need in the x and y dimensions
xdim = int(round((smax-smin)/0.1+1,0))
ydim = int(round((tmax-tmin)+1,0))
 
# Create empty grid of zeros
dens = np.zeros((ydim,xdim))
 
# Create temp and salt vectors of appropiate dimensions
ti = np.linspace(1,ydim-1,ydim)+tmin
si = np.linspace(1,xdim-1,xdim)*0.1+smin
 
# Loop to fill in grid with densities
for j in range(0,int(ydim)):
    for i in range(0, int(xdim)):
        dens[j,i]=gsw.rho(si[i],ti[j],0)

# Substract 1000 to convert to sigma-t
dens = dens - 1000


    
# plot T and S diagrams first with distance from coast and then with other variables
fig, ax = plt.subplots(figsize=(8.5,7))
markersize=40

ax.set_xlim(32,37)
ax.set_ylim(8,30)
ax.set_xlabel('Salinity')
ax.set_ylabel('Temperature (C)')

CS = ax.contour(si,ti,dens, linestyles='dashed', colors='k', alpha=0.5)
ax.clabel(CS, fontsize=12, inline=1, fmt='%1.1f') # Label every second level
# ellipse = Ellipse(xy=(35.88, 28.05), width=0.5, height=1.2, angle=-20,
#                 edgecolor='k', fc='None', ls='--', lw=2)
# ax.add_patch(ellipse)

# im = ax.scatter(np.concatenate(vmp_sal_list), np.concatenate(vmp_temp_list), c=np.concatenate(vmp_chla_list), vmin=-0.5,vmax=0.5,alpha=0.4)#c=np.concatenate(vmp_dt_list), vmin=0,vmax=365, cmap='twilight')

temp_est = provide_gs_temp_threshold(np.concatenate(vmp_dt_list))
sal_est = provide_gs_sal_threshold(np.concatenate(vmp_dt_list))

im = ax.scatter(np.concatenate(vmp_sal_list)-(sal_est-annual_sal_mean), np.concatenate(vmp_temp_list)-(temp_est-annual_temp_mean),c=gs_water.astype(np.int), cmap='bwr',)


fig.colorbar(im)
ax.set_title("Normalized S and T")
fig.savefig('normed_ts_diagram_allprofiles.png',dpi=300)
    
# ax.legend()
# fig.savefig('figs/simple_t_s_diagram_all_transects_with_divisions.png',dpi=300)

# plot T and S diagrams first with distance from coast and then with other variables
fig, ax = plt.subplots(figsize=(8.5,7))
markersize=40

ax.set_xlim(32,37)
ax.set_ylim(8,30)
ax.set_xlabel('Salinity')
ax.set_ylabel('Temperature (C)')

CS = ax.contour(si,ti,dens, linestyles='dashed', colors='k', alpha=0.5)
ax.clabel(CS, fontsize=12, inline=1, fmt='%1.1f') # Label every second level
# ellipse = Ellipse(xy=(35.88, 28.05), width=0.5, height=1.2, angle=-20,
#                 edgecolor='k', fc='None', ls='--', lw=2)
# ax.add_patch(ellipse)

# im = ax.scatter(np.concatenate(vmp_sal_list), np.concatenate(vmp_temp_list), c=abs(np.concatenate(vmp_dt_list)-225), vmin=0,vmax=225, cmap=cmocean.cm.ice,alpha=0.4)
im = ax.scatter(np.concatenate(vmp_sal_list)[gs_water.astype(np.int) ==1], np.concatenate(vmp_temp_list)[gs_water.astype(np.int) ==1], c=abs(np.concatenate(vmp_dt_list)-225)[gs_water.astype(np.int) ==1], cmap='jet', marker='x',vmin=0,vmax=225)
im = ax.scatter(np.concatenate(vmp_sal_list)[gs_water.astype(np.int) ==0], np.concatenate(vmp_temp_list)[gs_water.astype(np.int) ==0], c=abs(np.concatenate(vmp_dt_list)-225)[gs_water.astype(np.int) ==0], cmap='jet', marker='^',vmin=0,vmax=225)
fig.colorbar(im)
ax.set_title("Raw S and T")
fig.savefig('ts_diagram_allprofiles_DOY.png',dpi=300)
    


3D view of T-S diagram

In [ ]:
plt.hist(np.concatenate(vmp_chla_list))

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(np.concatenate(vmp_temp_list), np.concatenate(vmp_sal_list), abs(np.concatenate(vmp_dt_list)-225),c=gs_water.astype(np.int), cmap='bwr')

ax.set_ylabel('Sal (PSU)')
ax.set_xlabel('Temp (C)')
ax.set_zlabel('Days from Aug 15')
# fig.savefig('chla_across_TS_DOY.png',dpi=300)

Looking at ek backscatter seasonality in the GS water

In [ ]:
profiles['gs_sal_thresh'] = provide_gs_sal_threshold(profiles['daysfromaug15'], buffer=0.25)

In [ ]:
profiles.head()

In [ ]:
profiles['daysfromaug15'] = abs(profiles.datetime.dt.dayofyear-225)

In [ ]:
profiles_gs = profiles[profiles['salinity (PSU)'] > profiles['gs_sal_thresh']]
profiles_coastal = profiles[profiles['salinity (PSU)'] < profiles['gs_sal_thresh']]

In [ ]:
plt.scatter(profiles[profiles['depth (dBars)'] > -35]['daysfromaug15'], profiles[profiles['depth (dBars)'] > -35]['ekbackscatter'], c=profiles[profiles['depth (dBars)'] > -35]['temp (C)'],cmap=cmocean.cm.thermal)

In [ ]:
plt.hist(profiles_gs[profiles_gs['depth (dBars)'] > -35]['chla (ppb)'],bins=500)
plt.xlim(-1,2)

In [ ]:
depth = -45

In [ ]:
profiles_gs.groupby('daysfromaug15').mean().index

In [ ]:
plt.scatter(profiles_gs[profiles_gs['depth (dBars)'] > depth].groupby('daysfromaug15').mean().index, profiles_gs[profiles_gs['depth (dBars)'] > depth].groupby('daysfromaug15').mean()['chla (ppb)'], 
            c=profiles_gs[profiles_gs['depth (dBars)'] > depth].groupby('daysfromaug15').mean()['chla (ppb)'],vmin=-.5,vmax=.8,cmap=cmocean.cm.algae)
plt.ylim(-.25,.5)
plt.xlim(0,225)

In [ ]:
plt.scatter(profiles_gs[profiles_gs['depth (dBars)'] > depth].groupby('daysfromaug15').mean().index, profiles_gs[profiles_gs['depth (dBars)'] > depth].groupby('daysfromaug15').mean()['ekbackscatter'], 
            c=profiles_gs[profiles_gs['depth (dBars)'] > depth].groupby('daysfromaug15').mean()['chla (ppb)'],vmin=-.5,vmax=.8,cmap=cmocean.cm.algae)
plt.xlim(0,225)

In [ ]:
plt.scatter(profiles_coastal[profiles_coastal['depth (dBars)'] > depth]['daysfromaug15'], profiles_coastal[profiles_coastal['depth (dBars)'] > depth]['ekbackscatter'], c=profiles_coastal[profiles_coastal['depth (dBars)'] > depth]['chla (ppb)'],vmin=-.5,vmax=.8,cmap=cmocean.cm.algae)
plt.xlim(0,225)

Combining the S and T GS Frontal division

In [ ]:
tester = np.array(tester)

plot just the gs water trends

In [ ]:
gs_water

fig, ax = plt.subplots()

im = ax.scatter(abs(np.concatenate(vmp_dt_list)-225)[~tester],vmp_chla_clean[~tester], c=np.concatenate(vmp_sal_list)[~tester], cmap=cmocean.cm.haline,facecolors='none',)
fig.colorbar(im)
# abs(np.concatenate(vmp_dt_list) -180)

In [ ]:
gs_water

fig, ax = plt.subplots()

im = ax.scatter(abs(np.concatenate(vmp_dt_list)-225)[gs_water],vmp_chla_clean[gs_water], facecolors='none',)#c=np.concatenate(vmp_temp_list)[gs_water], cmap=cmocean.cm.thermal)
fig.colorbar(im)
# abs(np.concatenate(vmp_dt_list) -180)

In [ ]:
gs_water

fig, ax = plt.subplots()

im = ax.scatter(np.concatenate(vmp_temp_list)[gs_water],np.concatenate(vmp_sal_list)[gs_water], c=vmp_chla_clean[gs_water], cmap=cmocean.cm.algae, vmin=-0.25,vmax=0.25)
fig.colorbar(im)
# abs(np.concatenate(vmp_dt_list) -180)

In [ ]:
gs_water

fig, ax = plt.subplots()

im = ax.scatter(np.concatenate(vmp_temp_list)[~gs_water],vmp_chla_clean[~gs_water], c=vmp_chla_clean[~gs_water], cmap=cmocean.cm.algae, vmin=-0.25,vmax=0.25)
fig.colorbar(im)
# abs(np.concatenate(vmp_dt_list) -180)

In [ ]:
gs_water

fig, ax = plt.subplots()

im = ax.scatter(np.concatenate(vmp_sal_list)[~gs_water],vmp_chla_clean[~gs_water], c=np.concatenate(vmp_temp_list)[~gs_water], cmap=cmocean.cm.algae)
fig.colorbar(im)
# abs(np.concatenate(vmp_dt_list) -180)

In [ ]:
import cmocean

In [ ]:
fig, ax = plt.subplots()

im = ax.scatter(abs(np.concatenate(vmp_dt_list)-225),vmp_chla_clean, c=np.concatenate(vmp_sal_list), cmap=cmocean.cm.haline)
fig.colorbar(im)
# abs(np.concatenate(vmp_dt_list) -180)

In [ ]:
fig, ax = plt.subplots()

# im = ax.scatter(abs(dates_x_sst-225),sst_mean.gs_sst-273, c=dates_x_sst)
temp_est = provide_gs_temp_threshold(dates_x_sst)
im = ax.scatter(dates_x_sst,sst_mean.gs_sst-273-temp_est, c=dates_x_sst)

fig.colorbar(im)
# abs(np.concatenate(vmp_dt_list) -180)

In [ ]:
fig, ax = plt.subplots(figsize=(8,7))
im = ax.scatter(np.concatenate(vmp_sal_list), np.concatenate(vmp_temp_list), c=np.concatenate(vmp_chla_list), vmin=-0.5,vmax=0.5)
fig.colorbar(im)
plt.show()
fig, ax = plt.subplots(figsize=(8,7))
im = ax.scatter(np.concatenate(vmp_sal_list), np.concatenate(vmp_temp_list), c=np.concatenate(vmp_dt_list), vmin=0,vmax=365, cmap='twilight')
fig.colorbar(im)

In [ ]:
echo_reg

In [ ]:
interp_sst_list = np.array(interp_sst_list)
interp_current_list = np.array(interp_current_list)
interp_echo_list = np.array(interp_echo_list)

In [ ]:
# datetimes = [datetime.strptime(f.split('_')[2][4:-5], '%m%dT%H%S') for f in filenames]

datetimes = [datetime.strptime(f.split('_')[2][:-5], '%Y%m%dT%H%S') for f in filenames]

In [ ]:
datetimes_subset = [datetimes[i] for i in tentative_good_idxs]

In [ ]:
datetimes_subset = [datetimes[i] for i in front_transects[:32]]

In [ ]:
len(datetimes_subset)

In [ ]:
np.arange(0,len(datetimes_subset),1)

In [ ]:
np.nan_to_num(interp_echo_list,0).shape

In [ ]:
for idx, x in enumerate(sal_front_list):
    if len(x) > 0:
        

In [ ]:
nrows = 3
ncols = 5
Z = np.nan_to_num(interp_echo_list,0)
x = np.arange(1200 + 1)
y = np.arange(49 + 1)

fig, ax = plt.subplots()
ax.pcolormesh(x, datetimes_subset, Z, shading='none', vmin=Z.min(), vmax=Z.max())

In [ ]:
plt.hist(interp_current_list.flatten())
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,9))
z = np.nan_to_num(interp_current_list,0)
z = np.ma.masked_array(z, z < 1)
# ax.pcolormesh(x, datetimes_subset, z, cmap='viridis',vmin=0,vmax=250)
im = ax.pcolormesh(z, cmap='bwr',vmin=0,vmax=200)
fig.colorbar(im)

ax.axvline(600,c='k',ls='--')
ax.set_yticklabels([])
for i in range(len(datetimes_subset)):
    ax.text(-130, i, datetimes_subset[i].strftime("%d - %m - %y"))
    
ax.set_xlabel('Dekameters (front is located at 600 dm)')
    
# plt.savefig('surface_current_speed_front_adcp.png',dpi=300)

In [ ]:
len(interp_sst_list)

In [ ]:
len(datetimes_subset)

In [ ]:
fig, ax = plt.subplots(figsize=(20,9))
z = np.nan_to_num(interp_sst_list,0)
z = np.ma.masked_array(z, z < 1)
im = ax.pcolormesh(z, cmap='inferno',vmin=12,vmax=30)
fig.colorbar(im)

ax.axvline(600,c='k',ls='--')
ax.set_yticklabels([])
for i in range(len(datetimes_subset)):
    ax.text(-130, i, datetimes_subset[i].strftime("%d - %m - %y"))
    
ax.set_xlabel('Dekameters (front is located at 600 dm)')
        
# plt.savefig('surface_sst_front_adcp.png',dpi=300)

In [ ]:
np.array(interp_echo_list).shape

In [ ]:
from scipy import ndimage
# a Gaussian filter with a standard deviation of 10
[ndimage.gaussian_filter1d(line, 10) for line in interp_echo_list]

In [ ]:
fig, ax = plt.subplots(figsize=(20,9))
z = np.nan_to_num([ndimage.gaussian_filter1d(line, 5) for line in interp_echo_list],0)
z = np.ma.masked_array(z, z < 1)
ax.pcolormesh(z, cmap='jet',vmin=95,vmax=120)

ax.axvline(600,c='k',alpha=0.2)

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))
ax.scatter(datetimes_subset,np.nanmean(interp_echo_list[:,450:600],axis=1),c='red',label='Hot Side')
ax.scatter(datetimes_subset,np.nanmean(interp_echo_list[:,750:900],axis=1),c='blue',label='Cold Side')
ax.legend()
ax.set_ylabel('Backscatter (sV)')
ax.set_xlabel('Date')

ax.set_title('Backscatter from top 10m')
fig.autofmt_xdate(bottom=0.2, rotation=30, ha='right')

# plt.savefig('echo_response_full_dataset.png',dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))
ax.scatter(datetimes_subset,np.nanmean(interp_sst_list[:,450:600],axis=1),c='red')
ax.scatter(datetimes_subset,np.nanmean(interp_sst_list[:,600:750],axis=1),c='blue')

ax.set_ylabel('SST (C)')
ax.set_xlabel('Date')

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))
ax.scatter(datetimes_subset,np.nanmean(interp_current_list[:,450:600],axis=1),c='red')
ax.scatter(datetimes_subset,np.nanmean(interp_current_list[:,600:750],axis=1),c='blue')

ax.set_ylabel('current speed (cm/s)')
ax.set_xlabel('Date')

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))
ax.scatter(datetimes_subset,np.nanmean(interp_current_list[:,450:600]-interp_current_list[:,600:750],axis=1),c='red')
# ax.scatter(datetimes_subset,np.nanmean(interp_current_list[:,600:750],axis=1),c='blue')

ax.set_ylabel('current speed (cm/s)')
ax.set_xlabel('Date')

In [ ]:
# make an array with x as distance and y as sst

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))


# for i in range(len(dist_list)):
for i in range(10):
    dist = dist_list[i]
    echo = echo_list[i]
    front_location = front_location_list[i]
    echo_reg = interp_echo_list[i]
    
    dist_centered = dist - dist[front_location]
    ax.plot(np.arange(-12000,12000, grid_spacing),echo_reg,alpha=0.3)
    
# ax.plot(np.arange(-12000,12000, grid_spacing),np.nanmean(interp_echo_list,axis=0), color='black',alpha=1, lw=3, label='mean echo')
# ax.plot(np.arange(-12000,12000, grid_spacing),np.nanstd(interp_echo_list,axis=0)+np.nanmean(interp_echo_list,axis=0), color='black',alpha=.5, lw=2,label='std deviation')
# ax.plot(np.arange(-12000,12000, grid_spacing),np.nanmean(interp_echo_list,axis=0)-np.nanstd(interp_echo_list,axis=0), color='black',alpha=.5, lw=2)

ax.set_ylabel('echo (sV)')
ax.set_xlabel('distance from front (m, negative is offshore)')


ax.axvline(0,c='k',ls='--')
ax.set_xlim(-6000,6000)

# fig.savefig('adcp_sst_across_front.png',dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))


for i in range(len(dist_list)):
    dist = dist_list[i]
    sst = sst_list[i]
    front_location = front_location_list[i]
    sst_reg = interp_sst_list[i]
    
    dist_centered = dist - dist[front_location]
    ax.plot(np.arange(-12000,12000, grid_spacing),sst_reg,alpha=0.3)
    
ax.plot(np.arange(-12000,12000, grid_spacing),np.nanmean(interp_sst_list,axis=0), color='black',alpha=1, lw=3, label='mean SST')
ax.plot(np.arange(-12000,12000, grid_spacing),np.nanstd(interp_sst_list,axis=0)+np.nanmean(interp_sst_list,axis=0), color='black',alpha=.5, lw=2,label='std deviation')
ax.plot(np.arange(-12000,12000, grid_spacing),np.nanmean(interp_sst_list,axis=0)-np.nanstd(interp_sst_list,axis=0), color='black',alpha=.5, lw=2)

ax.set_ylabel('SST (C)')
ax.set_xlabel('distance from front (m, negative is offshore)')


ax.axvline(0,c='k',ls='--')
ax.set_xlim(-6000,6000)

# fig.savefig('adcp_sst_across_front.png',dpi=300)
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 16})


In [ ]:
current_reg[600]

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))


for i in range(len(dist_list)):
    dist = dist_list[i]
    sst = sst_list[i]
    front_location = front_location_list[i]
    current_reg = interp_current_list[i]
    
    dist_centered = dist - dist[front_location]
    ax.plot(np.arange(-12000,12000, grid_spacing),current_reg,alpha=0.2)
    
ax.plot(np.arange(-12000,12000, grid_spacing),np.nanmean(interp_current_list,axis=0), color='black',alpha=1, lw=3, label='mean current')
ax.plot(np.arange(-12000,12000, grid_spacing),np.nanstd(interp_current_list,axis=0)+np.nanmean(interp_current_list,axis=0), color='black',alpha=.5, lw=2, label='std deviation')
ax.plot(np.arange(-12000,12000, grid_spacing),np.nanmean(interp_current_list,axis=0)-np.nanstd(interp_current_list,axis=0), color='black',alpha=.5, lw=2)

ax.set_ylabel('current speed (m/s)')
ax.set_xlabel('distance from front (m, negative is offshore)')

ax.legend()
ax.axvline(0,c='k',ls='--')
ax.set_xlim(-6000,6000)
# fig.savefig('adcp_current_speed_across_front.png',dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))


for i in range(len(dist_list)):
    dist = dist_list[i]
    sst = sst_list[i]
    front_location = front_location_list[i]
    
    dist_centered = dist - dist[front_location]
    ax.plot(dist_centered,sst,alpha=0.5)
ax.axvline(0,c='k',ls='--')
ax.set_xlim(-6000,6000)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))


for i in range(len(dist_list)):
    dist = dist_list[i]
    sst = sst_list[i]
    front_location = front_location_list[i]
    
    dist_centered = dist - dist[front_location]
    ax.plot(dist_centered,sst/sst[front_location],alpha=0.5)
ax.set_xlim(-6000,6000)
plt.show()

In [ ]:
sst

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))


for i in range(len(dist_list)):
    dist = dist_list[i]
    sst = sst_list[i]
    front_location = front_location_list[i]
    
    dist_centered = dist - dist[front_location]
    ax.plot(dist_centered,sst/sst[front_location],alpha=0.5)
ax.set_xlim(-6000,6000)
ax.set_ylim(.8,1.2)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))


for i in range(len(dist_list)):
    dist = dist_list[i]
    current = current_list[i]
    front_location = front_location_list[i]
    
    dist_centered = dist - dist[front_location]
    ax.plot(dist_centered,current/current[front_location],alpha=0.5)
ax.axvline(0,c='k',ls='--')
ax.set_xlim(-6000,6000)
ax.set_ylim(0,2.2)
plt.show()